# Processetti
<br>

## Librerie

>   Importazione

In [1]:
import json
import pprint
from collections import Counter
from operator import itemgetter
import csv
from dotmap import DotMap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

-----------------

## Funzioni locali

In [2]:
# pulizia delle stringhe

def clean_up_str(string):
    a = ''
    if string:
        a = string.replace('\xa0', '').replace('  ', ' ').strip()
    return a  

In [3]:
# Funzione di elminazione delle dupplicazioni in leggenda
##  https://stackoverflow.com/questions/19385639/duplicate-items-in-legend-in-matplotlib

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique))

In [4]:
# calcolo dell'anno (medio)

def date(r_col):

    lista=[]
    for v in r_col['values']:
        if v.value.timePrimitive:
            lista.append(int(v.value.timePrimitive.label[:4]))
            
    if len(lista)>=1:
        return int(np.rint(np.mean(lista)))
    else:
        return np.nan



In [5]:
# sintesi time-spen (anno medio: begin, at some time, end)

def datatio(r_col_begin, r_col_someTime, r_col_end, element):

    begin, some_time, end = date(r_col_begin), date(r_col_someTime), date(r_col_end)
    element+= [begin, some_time, end]

    return(element)

-------------

## Preparazione dei dataset


### Overview Processetti



#### Appertura e ispezione dei file 

In questa sezione i dati vengono ispezionati una prima volta. Vengono importati gli _output_ scaricati da Geovistory 

I file contengono il risultato di un'interrogazione fatta in Geovistory, una tabella del dominio Analysis, con lo stesso nome, fatta nella data indicata nel nome del file. Se si vuole utilizzare un nuovo file, contenente nuovi dati, va scaricarlo da Geovistory, caricato nella cartella *geovistory_query_exports* e indicare l'indirizzo corretto nella cella seguente

In [6]:
file = 'geovistory_query_exports/nb_overwiev_processetti_20220510.json'

In [7]:
# Aprire il file e attribuirlo alla variabile *data* in forma di dizionario Python

with open(file, encoding='utf-8') as json_file:
    data_pro = json.load(json_file)

In [8]:
# Creare un oggetto DotMap con lo scopo di facilitare il lavoro sul file JSON
dmp = DotMap(data_pro)

In [9]:
# Creare una variabile contenente le righe
dmp_r = dmp.rows

> Riprendi descrizione
### Trasformazione e messa in piano dei dati

È necessario disporre di dati trasformati e misurati (sopprimendo in particolare i valori multipli per una singola variabile) per poter effettuare le analisi.

La seguente procedura è stata applicata per creare una nuova tabella:
- se esistevano più anni (per esempio l'anno di nascita), è stata scelta la media
- se esistevano più occupazioni, tutte sono state mantenute e separate con un segno "#".
- se esistevano più tipi di processetto, tutti sono stati mantenuti e separati con un segno "#".


Qui sotto trovate un estratto di questa nuova tabella. Questa nuova tabella viene esportata automaticamente come documento csv. Puoi accedere e scaricare questo documento nel tuo jupyter-binder. Puoi aprirlo in Excel (via csv-import) e analizzarlo anche lì. Questo potrebbe aiutare a capire la struttura della tabella appena creata.


> Riprendi descrizione

In [10]:
data_pro['columns']

{'col_0': 'Volume',
 'col_1': 'Unità archivistiche',
 'col_2': 'Processetto di',
 'col_3': 'Motivation Type',
 'col_4': 'date',
 'col_5': 'Union',
 'col_6': 'Person (Richiedente)'}

In [11]:
### Processetti list = psl

psl=[['volume', 'section', 'number_archival_units', 'pk_processetto', 'enquiry','pk_person', 'motivation_type',
 'start_date','end_date', 'anno', 'id_union', 'number_union']]


for r in dmp_r:
    element = []
    # volume
    element += ['-'.join([v.entity_label for v in r.col_0.entities]) if len(r.col_0.entities)>=1 else np.nan]   

    # sezione (unità archivistica)
    element += [clean_up_str(r.col_1.entities[0].entity_label)]

    # numero di section per enquiry 
    element += [len(r.col_1.entities)]

    # enquiry 
    element += [r.col_2.entity.pk_entity, clean_up_str(r.col_2.entity.entity_label)]

    # pk_person     ## Integrazione (08/08/2022) -- garantirà un miglior allineamento -- la query in Geovistory è già stata integrata
    element += [r.col_6.entities[0].pk_entity if r.col_6.entities else np.nan]
   
    # motivation type
    element += [clean_up_str('; '.join([v.entity_label for v in r.col_3.entities])) if len(r.col_3.entities)>=1 else np.nan]

    # arco e anno processetto
    valor = []
    if len(r.col_4['values']) == 0:
        element += [np.nan, np.nan, np.nan]
    else:
        for v in  r.col_4['values']:
            valor.append(int(v.value.timePrimitive.label[:4]))
        element += [r.col_4['values'][0]['value'].timePrimitive.label[:10], r.col_4[
            'values'][len(r.col_4['values'])-1]['value'].timePrimitive.label[:10], round(np.mean(valor))]

    # union
    if len(r.col_5.entities)==1:
        element += [r.col_5.entities[0].pk_entity, len(r.col_5.entities)]
    elif len(r.col_5.entities)>1:
        element += ['-'.join([str(v.pk_entity) for v in r.col_5.entities]) if len(r.col_5.entities)>=1 else np.nan, len(r.col_5.entities)]  
    else:
        element += [np.nan, 0]
        
    psl.append(element)
    

processetti = pd.DataFrame(psl[1:], columns=psl[0])  
processetti.head()



,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),1006422.0,Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",999599.0,Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",1006794.0,Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),1006856.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),1007216.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0


In [12]:
def processetti_generator(file):
    file = file
    with open(file, encoding='utf-8') as json_file:
        data_processetti = json.load(json_file)
    dmprocessetti = DotMap(data_processetti)
    dmprocesetti_rows = dmprocessetti.rows


    ### Processetti list = psl

    psl=[['volume', 'section', 'number_archival_units', 'pk_processetto', 'enquiry','pk_person', 'motivation_type',
 'start_date','end_date', 'anno', 'id_union', 'number_union']]

    # r for row
    for r in dmprocesetti_rows:
    
        element = []
        # volume
        element += ['-'.join([v.entity_label for v in r.col_0.entities]) if len(r.col_0.entities)>=1 else np.nan]   

        # sezione (unità archivistica)
        element += [clean_up_str(r.col_1.entities[0].entity_label)]

        # numero di section per enquiry 
        element += [len(r.col_1.entities)]

        # enquiry 
        element += [r.col_2.entity.pk_entity, clean_up_str(r.col_2.entity.entity_label)]

        # pk_person
        element += [r.col_6.entities[0].pk_entity if r.col_6.entities else np.nan]

        # motivation type
        element += [clean_up_str('; '.join([v.entity_label for v in r.col_3.entities])) if len(r.col_3.entities)>=1 else np.nan]

        # arco e anno processetto
        valor = []
        if len(r.col_4['values']) == 0:
            element += [np.nan, np.nan, np.nan]
        else:
            for v in  r.col_4['values']:
                valor.append(int(v.value.timePrimitive.label[:4]))
            element += [r.col_4['values'][0]['value'].timePrimitive.label[:10], r.col_4[
                'values'][len(r.col_4['values'])-1]['value'].timePrimitive.label[:10], round(np.mean(valor))]

        # union
        if len(r.col_5.entities)==1:
            element += [r.col_5.entities[0].pk_entity, len(r.col_5.entities)]
        elif len(r.col_5.entities)>1:
            element += ['-'.join([str(v.pk_entity) for v in r.col_5.entities]) if len(r.col_5.entities)>=1 else np.nan, len(r.col_5.entities)]  
        else:
            element += [np.nan, 0]
            
        psl.append(element)
    

    return pd.DataFrame(psl[1:], columns=psl[0])  

In [13]:
# processetti = processetti_generator('geovistory_query_exports/nb_overwiev_processetti_20220510.json')
processetti

,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),1006422.0,Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",999599.0,Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",1006794.0,Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),1006856.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),1007216.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,EM 05,"Processetto di Gaspare Forzanini (da Bassano, ...",1,998606,"Gaspare Forzanini (da Bassano, calegher)",998390.0,Stato libero di “foresto”,1598-12-18,1598-12-18,1598.0,NaN,0
3793,EM 04,Processetto di Cristina vedova di Nicola De Ga...,1,998615,Cristina vedova di Nicola De Gaspari (da Venezia),998555.0,Vedovanza,1597-05-20,1597-05-20,1597.0,998630,1
3794,EM 04,Processetto di Giovanni Maria di Antonio (da L...,1,998913,"Giovanni Maria di Antonio (da Lovere, tesser d...",998798.0,Vedovanza,1597-05-21,1597-05-21,1597.0,998922,1
3795,EM 05,Processetto di Andrea di Angelo Pasetto (da Ma...,1,999218,"Andrea di Angelo Pasetto (da Magno, erbariol)",998838.0,Stato libero di “foresto”,1598-12-21,1598-12-21,1598.0,NaN,0


In [14]:
processetti.describe(exclude=np.number)

,volume,section,enquiry,motivation_type,start_date,end_date,id_union
count,3797,3797,3797,3792,3788,3788,616
unique,29,3695,3782,12,2237,2257,613
top,EM 07,Processetto di Margherita di Matteo Paulini e ...,(no label),Stato libero di “foresto”,1632-02-21,1632-02-21,1079309
freq,503,3,11,3009,12,12,2


> Da sviluppare sulla base delle esigenze di ricerca

### Richiedenti

In [15]:
file = 'geovistory_query_exports/nb_richiedenti_20220510.json'

with open(file, encoding='utf-8') as json_file:
    data_ric = json.load(json_file)

dmr = DotMap(data_ric)

dmr_r = dmr.rows 

In [16]:
data_ric['columns']

{'col_0': 'PK_Processetto_di',
 'col_1': 'Richiedenti',
 'col_2': 'Genere',
 'col_3': 'Origine',
 'col_4': 'Età',
 'col_5': 'Professione',
 'col_6': 'Localisation'}

In [17]:
# Conteggio del numero max di pk_processetti per persona

conteggio = []; conteggio+= [len(r.col_0.entities) for r in dmr_r]; max(conteggio)

2

In [18]:
# Richiedenti list = rl

rl = [['pk_processetto',  'pk_person', 'person', 'qualità', 'genere', 'origini', 'anno_nascita', 'professioni' ]]


for r in dmr_r:
    for el in r.col_0.entities:
        element = []
        element += [el.pk_entity]

        # pk_person, person, qualità, genere
        element += [r.col_1.entity.pk_entity, clean_up_str(r.col_1.entity.entity_label), 'richiedente', (
            r.col_2.entities[0].entity_label if len(r.col_2.entities)>0 else np.nan)]
        
        # origini
        element += [clean_up_str('; '.join(e.entity_label for e in r.col_3.entities)) if len(r.col_3.entities)>=1 else np.nan]

        # Anno di nascita 
        element += [date(r.col_4)]
        
        # professioni
        element += ['; '.join(e.entity_label for e in r.col_5.entities) if len(r.col_5.entities)>=1 else np.nan]
        
        rl.append(element)


richiedenti = pd.DataFrame(rl[1:], columns=rl[0])
richiedenti['età'] = richiedenti.merge(processetti[['pk_processetto', 'anno']], on='pk_processetto', how='left')['anno'] - richiedenti['anno_nascita']
richiedenti = richiedenti[['pk_processetto', 'pk_person', 'person', 'qualità', 'età', 'anno_nascita', 'genere', 'origini', 'professioni']]; richiedenti.head()



,pk_processetto,pk_person,person,qualità,età,anno_nascita,genere,origini,professioni
0,1006519,1006422,Giovanni di Michele (da Cessalto),richiedente,21.0,1576.0,Maschile,Cessalto,NaN
1,1006883,1006794,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",richiedente,21.0,1576.0,Maschile,Bergamo,cogo; tesser da panni
2,1007147,1006856,Orsola di Angelo Collotis (da Visinale),richiedente,26.0,1572.0,Femminile,Visinale,masser
3,1007355,1007216,Ortensia di Leonardo Verona (da Venzone),richiedente,25.0,1573.0,Femminile,Venzone,masser
4,1007563,1007388,"Vincenzo di Battista Betocchi (da Gandino, cas...",richiedente,26.0,1572.0,Maschile,Gandino,casariol


In [19]:
richiedenti.describe(exclude=np.number)

,person,qualità,genere,origini,professioni
count,3429,3429,3422,3126,2314
unique,3424,1,2,952,409
top,"Pasqualina di Giorgio (mariner), vedova di Dom...",richiedente,Maschile,Venezia IT,masser
freq,2,3429,2560,126,204


In [20]:
def persons_generator(file, quality=''):
    file = file
    quality = quality
    with open(file, encoding='utf-8') as json_file:
        data_ric = json.load(json_file)

    dm = DotMap(data_ric)
    dm_r = dm.rows  

    if ('tes' or 'wit') in quality:
        quality='testimone'
    elif ('ric' or 'req' or 'app' or 'inv') in quality:
        quality='richiedente'
    elif ('def' or 'dec') in quality:
        quality='defunto'
    

    if (quality == 'testimone') or (quality== 'richiedente'):

        # Richiedenti list = rl
        lista = [['pk_processetto',  'pk_person', 'person', 'qualità', 'genere', 'origini', 'professioni', 'anno_nascita']]


        for r in dm_r:
            for el in r.col_0.entities:
                element = []
                element += [el.pk_entity]

                # pk_person, person, qualità, genere
                element += [r.col_1.entity.pk_entity, clean_up_str(r.col_1.entity.entity_label), quality, (
                    r.col_2.entities[0].entity_label if len(r.col_2.entities)>0 else np.nan)]
                
                # origini
                element += [clean_up_str('; '.join(e.entity_label for e in r.col_3.entities)) if len(r.col_3.entities)>=1 else np.nan]

                # professioni
                element += ['; '.join(e.entity_label for e in r.col_5.entities) if len(r.col_5.entities)>=1 else np.nan]
                
                # Anno di nascita 
                element += [date(r.col_4)]
                
                
            
                
                lista.append(element)

    elif quality == 'defunto':

        lista = [['pk_person', 'person', 'qualità', 'genere', 'origini', 'professioni', 'anno_nascita', 'anno_morte', 'luogo_morte', 'id_union', 'union_count', 'type_union', 'pk_person(partner)', 'partner', 'pk_processetto']]

        for r in dm_r:
            element=[]

            # pk_person, person, qualità, genere, origini, professioni
            element += [r.col_0.entity.pk_entity, clean_up_str(r.col_0.entity.entity_label), quality, (r.col_1.entities[0].entity_label if len(r.col_1.entities)>0 else np.nan), (
                ('; '.join(e.entity_label for e in r.col_2.entities)) if len(r.col_2.entities)>=1 else np.nan), ('; '.join(e.entity_label for e in r.col_4.entities) if len(r.col_4.entities)>=1 else np.nan)]

            # anno_nascita
            element += [date(r.col_3)]

            # anno_morte, luogo_morte
            element += [date(r.col_6), ('; '.join(e.entity_label for e in r.col_7.entities) if len(r.col_7.entities)>=1 else np.nan)]
            
            # union, conteggio_unioni, tipo unione
            element += ['-'.join([str(v.pk_entity) for v in r.col_8.entities]) if len(r.col_8.entities)>=1 else np.nan, len(r.col_8.entities), (
                '; '.join(e.entity_label for e in r.col_9.entities) if len(r.col_9.entities)>=1 else np.nan)]
            
            # pk_person(partner), partner
            ## riscrivi
            id_part, part = [], []
            for e in r.col_10.entities:
                if e.pk_entity != r.col_0.entity.pk_entity:
                    id_part.append(e.pk_entity)
                    part.append(clean_up_str(e.entity_label))
            element += [('-'.join(str(e) for e in id_part) if len(id_part)>=1 else np.nan), ('; '.join(part) if len(part)>=1 else np.nan)]
                
            # pk_processetto
            element += ['-'.join([str(v.pk_entity) for v in r.col_11.entities]) if len(r.col_11.entities)>=1 else np.nan]

            lista.append(element)  


    return  pd.DataFrame(lista[1:], columns=lista[0])
    


### Testimoni

In [21]:
file = 'geovistory_query_exports/nb_testimoni_20220510.json'

with open(file, encoding='utf8') as json_file:
    data_tes = json.load(json_file)

dmt = DotMap(data_tes)

dmt_r = dmt.rows 

In [22]:
print(data_tes['columns'])
dmt_r[0].col_1

{'col_0': 'PK_Processetto_di', 'col_1': 'Testimoni', 'col_2': 'Genere', 'col_3': 'Origine', 'col_4': 'Età', 'col_5': 'Professione', 'col_6': 'Localisation'}


DotMap(entity=DotMap(key='591_1002990', project=591, fk_class=21, pk_entity=1002990, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Giuseppe di Francesco Domiziano (da Fabriano, cimador da panni)', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00'), _ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [23]:
# Conteggio del numero max di pk_processetti per persona

conteggio = []; conteggio+= [len(r.col_0.entities) for r in dmt_r]; max(conteggio)

5

In [24]:
# Testimone list = rl

tl = [['pk_processetto',  'pk_person', 'person', 'qualità', 'genere', 'origini', 'anno_nascita', 'professioni' ]]


for r in dmt_r:
    for el in r.col_0.entities:
        element = []
        element += [el.pk_entity]

        # pk_person, person, qualità, genere
        element += [r.col_1.entity.pk_entity, clean_up_str(r.col_1.entity.entity_label), 'testimone', (
            r.col_2.entities[0].entity_label if len(r.col_2.entities)>0 else np.nan)]
        
        # origini
        element += [clean_up_str('; '.join(e.entity_label for e in r.col_3.entities)) if len(r.col_3.entities) else np.nan]

        # Anno di nascita 
        element += [date(r.col_4)]
        
        # professioni
        element += ['; '.join(e.entity_label for e in r.col_5.entities) if len(r.col_5.entities) else np.nan]
        
        tl.append(element)


testimoni = pd.DataFrame(tl[1:], columns=tl[0])
testimoni['età'] = testimoni.merge(processetti[['pk_processetto', 'anno']], on='pk_processetto', how='left')['anno'] - testimoni['anno_nascita']
testimoni = testimoni[['pk_processetto', 'pk_person', 'person', 'qualità', 'età', 'anno_nascita', 'genere', 'origini', 'professioni']]; testimoni.head()

,pk_processetto,pk_person,person,qualità,età,anno_nascita,genere,origini,professioni
0,1006729,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",testimone,53.0,1545.0,Maschile,Fabriano,cimador da panni
1,1006519,1006430,Pietro di Giovanni Maria (da Venezia),testimone,54.0,1543.0,Maschile,Venezia IT,NaN
2,1006519,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",testimone,26.0,1571.0,Maschile,Bergamo,vender vin
3,1006519,1006449,"Frizerio di Antonio Zuliani (da Venezia, pesca...",testimone,38.0,1559.0,Maschile,Venezia IT,NaN
4,1006883,1006806,"Ludovico di Ludovico Morando (da Bergamo, tess...",testimone,39.0,1558.0,Maschile,Bergamo,tesser da panni


In [25]:
testimoni.describe(exclude=np.number)

,person,qualità,genere,origini,professioni
count,7026,7026,7002,6405,5479
unique,6746,1,2,1028,524
top,"Battista di Giacomo (dal Cadore, forner)",testimone,Maschile,Venezia IT,mariner
freq,5,7026,6743,1024,359


### Defunti

In [26]:
file = 'geovistory_query_exports/nb_defunti_20220510.json'

with open(file, encoding='utf-8') as json_file:
    data_def = json.load(json_file)

dmd = DotMap(data_def)

dmd_r = dmd['rows']

In [27]:
print(data_def['columns'])
dmd_r[0].col_8

{'col_0': 'Defunto', 'col_1': 'Genere', 'col_2': 'Origine', 'col_3': 'Età', 'col_4': 'Professione', 'col_5': 'Localisation', 'col_6': 'Data_morte', 'col_7': 'Luogo_morte', 'col_8': 'Union', 'col_9': 'Tipo_unione', 'col_10': 'Partner', 'col_11': 'PK_Processetto_di'}


DotMap(entities=[DotMap(key='591_1007670', fk_type=837471, project=591, fk_class=633, pk_entity=1007670, time_span=DotMap(p81b=DotMap(calendar='gregorian', duration='1 month', julianDay=2303834)), fk_project=591, type_label='Matrimonio cattolico', class_label='Union', entity_type='teEn', last_second=199053935999, entity_label='Santa vedova di Battista di Nicola, Battista di Nicola', first_second=199051257600, tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')], _ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [28]:
# Defunti list = dl
#'pk_processetto', 'pk_person', 'person', 'qualità', 'età', 'anno_nascita', 'genere', 'origini', 'professioni',]]

dl = [['pk_person', 'person', 'qualità', 'genere', 'origini', 'professioni', 'anno_nascita', 'anno_morte', 'luogo_morte', 'id_union', 'union_count', 'type_union', 'pk_person(partner)', 'partner', 'pk_processetto']]

for r in dmd_r:
    element=[]

    # pk_person, person, qualità, genere, origini, professioni
    element += [r.col_0.entity.pk_entity, clean_up_str(r.col_0.entity.entity_label), 'defunto', (r.col_1.entities[0].entity_label if len(r.col_1.entities)>0 else np.nan), (
        ('; '.join(e.entity_label for e in r.col_2.entities)) if len(r.col_2.entities)>=1 else np.nan), ('; '.join(e.entity_label for e in r.col_4.entities) if len(r.col_4.entities)>=1 else np.nan)]

    # anno_nascita
    element += [date(r.col_3)]

    # anno_morte, luogo_morte
    element += [date(r.col_6), ('; '.join(e.entity_label for e in r.col_7.entities) if len(r.col_7.entities)>=1 else np.nan)]
    
    # union, conteggio_unioni, tipo unione
    element += ['-'.join([str(v.pk_entity) for v in r.col_8.entities]) if len(r.col_8.entities)>=1 else np.nan, len(r.col_8.entities), (
        '; '.join(e.entity_label for e in r.col_9.entities) if len(r.col_9.entities)>=1 else np.nan)]
    
    # pk_person(partner), partner
    ## riscrivi
    id_part, part = [], []
    for e in r.col_10.entities:
        if e.pk_entity != r.col_0.entity.pk_entity:
            id_part.append(e.pk_entity)
            part.append(clean_up_str(e.entity_label))
    element += [('-'.join(str(e) for e in id_part) if len(id_part)>=1 else np.nan), ('; '.join(part) if len(part)>=1 else np.nan)]
          
    # pk_processetto
    element += ['-'.join([str(v.pk_entity) for v in r.col_11.entities]) if len(r.col_11.entities)>=1 else np.nan]

    dl.append(element)

defunti = pd.DataFrame(dl[1:], columns=dl[0])
defunti.head()


          



,pk_person,person,qualità,genere,origini,professioni,anno_nascita,anno_morte,luogo_morte,id_union,union_count,type_union,pk_person(partner),partner,pk_processetto
0,1007544,Battista di Nicola,defunto,Maschile,NaN,paron de nave,1543.0,1595.0,NaN,1007670,1,Matrimonio cattolico,1007492,Santa vedova di Battista di Nicola,1007675
1,1007750,Angela (da Bergamo),defunto,Femminile,Territorio di Bergamo,NaN,NaN,1596.0,NaN,1007767,1,Matrimonio cattolico,1007715,Filippo Amis (da Bergamo),1007777
2,1013001,Francesco Bettinelli (curamer),defunto,Maschile,NaN,curamer,NaN,1590.0,S. Aponal,1026547,1,Matrimonio cattolico,1012979,Caterina Albrizzi vedova di Francesco Bettinel...,1026527
3,1032159,Marco Antonio di Francesco Galeotto,defunto,Maschile,Venezia IT,NaN,NaN,1590.0,S. Martin,1032511,1,Matrimonio cattolico,1032116,Bradamante di Francesco Canano (da Pordenone) ...,1032502
4,1038739,"Giovanni di Domenico (da Trieste, mariner)",defunto,Maschile,Trieste IT,mariner,NaN,1597.0,Zante,1038920,1,Matrimonio cattolico,1038725,Caterina di Antonio Manzoni vedova di Giovanni...,1038911


In [29]:
defunti.describe(exclude=np.number)

,person,qualità,genere,origini,professioni,luogo_morte,id_union,type_union,pk_person(partner),partner,pk_processetto
count,468,468,464,265,224,369,468,456,467,467,453
unique,468,1,2,143,115,216,468,4,467,467,453
top,Battista di Nicola,defunto,Maschile,Venezia IT,mariner,Venezia IT,1007670,Matrimonio cattolico,1007492,Santa vedova di Battista di Nicola,1007675
freq,1,468,347,44,49,16,1,449,1,1,1


---------------------

### Person name

In [30]:
file = 'geovistory_query_exports/nb_person_name_20220518.json'
with open(file, encoding='utf-8') as json_file:
    data_name = json.load(json_file)

dmpn = DotMap(data_name)
dmpn_r = dmpn.rows

In [31]:
print(data_name['columns'])
dmpn_r[21]

{'col_0': 'Entity Preview', 'col_1': 'cognome', 'col_2': 'Titolo di rispetto', 'col_3': 'Titolo professionale'}


DotMap(col_0=DotMap(entity=DotMap(key='591_1007723', project=591, fk_class=21, pk_entity=1007723, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Giulio di Giacomo Robaziri (da Bergamo, cerusico)', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')), col_1=DotMap(entities=[DotMap(key='591_1007736', fk_type=741575, project=591, fk_class=868, pk_entity=1007736, time_span=DotMap(), fk_project=591, type_label='Cognome', class_label='Person Appellation in a Language', entity_type='teEn', entity_label='Robaziri', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')]), col_2=DotMap(entities=[DotMap(key='591_1007731', fk_type=746080, project=591, fk_class=868, pk_entity=1007731, time_span=DotMap(), fk_project=591, type_label='Titolo di rispetto o di cerimonia', class_label='Person Appellation in a Language', entity_type='teEn', entity_label='ser', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')]), col_3=DotMap(entities=[]), _ipython_display_=Dot

In [32]:

def str_normalization(string):
    if type(string) == str and (string != ''):
        string = string.strip().replace('\xa0', '').replace('  ', ' ').lower()
    return string

# Person name list = pnl

pnl = [['pk_person', 'person', 'cognome', 'cognome_count', 'titolo_rispetto', 'titolo_profesisonale']]

for r in dmpn_r:
    element = []
    element += [r.col_0.entity.pk_entity, r.col_0.entity.entity_label, clean_up_str('; '.join(c.entity_label for c in r.col_1.entities)), len(r.col_1.entities) , str_normalization('; '.join(tr.entity_label for tr in r.col_2.entities)),
     str_normalization('; '.join(tp.entity_label for tp in r.col_3.entities))]
    pnl.append(element)

person_name = pd.DataFrame(pnl[1:], columns=pnl[0])
person_name.replace('', np.nan, inplace=True)




In [33]:
person_name.loc[(person_name.cognome_count>1) & (person_name.cognome.str.contains('(?i)domin|(?i)ser'))]

,pk_person,person,cognome,cognome_count,titolo_rispetto,titolo_profesisonale
57,1012149,"Ludovico di Faustino Faustini (da Pregasio, sp...",ser; Faustini,2,NaN,NaN
393,1054933,Marietta di Antonio Gastaldione (da Flagogna),Gastaldione; Domina,2,NaN,NaN
1762,1147200,"Nicolò di Pompeo Baldi (da Ghedi, tesser da pa...",ser; Baldi,2,NaN,NaN
2263,1249404,"Cristiano di Giacomo Forame (da Livinallongo, ...",Ser; Forame (de),2,NaN,NaN
3171,1464762,Santo di Luciano Zanieli (da Potenza),Zanieli; domino,2,NaN,NaN
3270,1475954,"Giacomo di Floriano Rivolini (dalla Carnia, te...",ser; Rivolini,2,NaN,NaN
3586,1506247,"Cristoforo di Retini Sivi (da Redona, barbier)",Sivi; domino,2,NaN,NaN
5556,1743295,Federico di Simone Zanchi (tesser da panni),Ser; Zanchi,2,NaN,NaN
6352,1807799,"Marsilio di Bernardino Fontana (veronese, murer)",Domino ; Fontana,2,NaN,NaN
6419,1811411,Nicolò di Giovanni Parmesan (da Almissa),Parmesan; Domino,2,NaN,NaN


In [34]:
person_name.loc[(person_name.cognome_count>1) & (person_name.cognome.str.contains('(?i)domin|(?i)ser')==False)]

,pk_person,person,cognome,cognome_count,titolo_rispetto,titolo_profesisonale
183,1039703,Margherita di Baldo Balzar (da Cerete Basso),Balzar; Balssar,2,NaN,NaN
282,1046945,"Ludovico di Giuseppe Brunelleschi (da Venezia,...",Brunelleschi; Burnelesco,2,domino,NaN
403,1055490,Francesco di Giovanni Giacomo Turchi (da Cellore),Turchi; Turchi (de),2,NaN,NaN
409,1055644,"Camillo di Bartolomeo Pivati (da Venezia, tess...",Pivati (de); Pivatti (de),2,NaN,NaN
475,1059661,Carlo Polini (bolognese),Pollinus; Polini,2,domino,NaN
...,...,...,...,...,...,...
10240,756015,Giovanna di Battista De Zuliani vedova di Batt...,De Zuliani; De Pizzolesi,2,NaN,NaN
10308,761131,Vincenzo di Antonio De Caballis (mariner),De Cavalli; De Caballis,2,NaN,NaN
10354,764640,"Carlo di Giovanni Riese (da Villach, interpr...",Riese (Ariss); Gigante,2,ser,NaN
10495,773122,Adriana Motta vedova di Domenico Carnaris (da ...,Motta; Carnaris,2,NaN,NaN


In [35]:
person_name['titolo_rispetto'].unique()

array([nan, 'ser', 'domina', 'domino', 'clarissimo', 'clarissima',
       'domino; domino', 'reverendo presbitero; domino',
       'domino; reverendo presbitero', 'vir honestissimus dominus',
       'clarissimo domino', 'clarissima domina', 'reverendo; domino',
       'messer', 'magnifico', 'magnifico domino',
       'illustrissimo e reverendissimo', 'domino; magnifico', 'reverendo',
       'reverendo presbitero', 'domino; clarissimo', 'ser; domino',
       'conte; clarissimo domino',
       'cittadino veneziano; magnifico domino', 'maestro',
       'domino/domina', 'illustrissimo', 'magnifica domina',
       'nobile veneto; clarissimo domino',
       'nobile vicentino; magnifico domino', 'cornari', 'messer dominus',
       'presbitero; domino', 'magnifico; reverendo presbitero', 'strenuo',
       'nobile domina', 'doomina', 'eccellentissimo domino',
       'eccellente domino',
       'ser; ventura di lorenzo sbotta (da gandino, laner)', 'nobile',
       'signora', 'eccellentissimo ; m

In [36]:
person_name.titolo_profesisonale.unique()

array([nan, 'maestro', 'prete', 'reverendo presbitero', 'presbitero',
       'patron', 'ser', 'magister', 'capitano',
       'dottotre (artium et medicinae doctor)', 'ambasciatore',
       'reverendo', 'reverendo pre'], dtype=object)

### Note: (fatto)

Includi nella query *__cognome__*, *__titolo di rispetto__* e *__titolo professionale__*
>   Integra codice  

----------------------

### Localisation

In [37]:
file = 'geovistory_query_exports/nb_loclaisation_20220512.json'

with open(file, encoding='utf-8') as json_file:
    data_loc = json.load(json_file)

dml = DotMap(data_loc)

dml_r = dml.rows

In [38]:
print(data_loc['columns'], '\n')

for r in dml_r:
    
    if len(r.col_8.entities)>0:
        print(r.col_8)
        break
    


{'col_0': 'Entity Preview', 'col_1': 'Person', 'col_2': 'Localisation', 'col_3': 'Begin earliest', 'col_4': 'Begin leterst', 'col_5': 'At some time within', 'col_6': 'End earliest', 'col_7': 'End leaterst', 'col_8': 'Occours before', 'col_9': 'Occours after', 'col_10': 'Type', 'col_11': 'Localisation_type', 'col_12': 'Geocoordinate'} 

DotMap(entities=[DotMap(key='591_1011924', fk_type=925607, project=591, fk_class=212, pk_entity=1011924, time_span=DotMap(p82=DotMap(calendar='gregorian', duration='1 month', julianDay=2305083)), fk_project=591, type_label='Abitazione e luogo di lavoro   ', class_label='Geographical Location of a Physical Thing', entity_type='teEn', last_second=199161849599, entity_label='Antonio di Antonio Murla (da Fonzaso, samiter)   , S. Geremia', first_second=199159171200, tmsp_last_modification='2022-05-06T06:41:13.83725+00:00')])


In [39]:
# Localisation list = ll

ll = [['id_localisation', 'pk_person', 'person', 'geographical_place', 'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time', 'end', 'occours_before', 'occours_after']]

for r in dml_r:
    element = []

    # id_localisatio, pk_person, person, geo_place + (id), 
    ## valuta la comodità di tenere separato l'ID geographical Place, geographical place, type, coordinate (x,y), localization type 
    element += [r.col_0.entity.pk_entity, r.col_1.entities[0].pk_entity, clean_up_str(r.col_1.entities[0].entity_label), (r.col_2.entities[0].entity_label + f' ({ r.col_2.entities[0].pk_entity})')  if len(
        r.col_2.entities)>0 else np.nan, r.col_10.entities[0].entity_label if len(r.col_10.entities)>=1 else np.nan]
    if len(r.col_12['values'])>0:    
        element += [r.col_12['values'][0].value.geometry.geoJSON.coordinates[1], r.col_12['values'][0].value.geometry.geoJSON.coordinates[0]]
    else:
        element +=[np.nan, np.nan]
    element += [r.col_11.entities[0].entity_label if len(r.col_11.entities)>0 else np.nan]

    # Begin
    element += [round(np.mean([int(r.col_3['values'][0].value.timePrimitive.label[:4]), int(r.col_4['values'][0].value.timePrimitive.label[:4])])) if (len(r.col_3['values'])>0) and (len(r.col_4['values'])>0) else int(
        r.col_3['values'][0].value.timePrimitive.label[:4]) if (len(r.col_3['values'])>0) else int(r.col_4['values'][0].value.timePrimitive.label[:4]) if len(r.col_4['values'])>0 else np.nan]
    
    # At some time within
    element += [int(r.col_5['values'][0].value.timePrimitive.label[:4]) if len(r.col_5['values'])>0 else np.nan]

    # End
    element += [round(np.mean([int(r.col_6['values'][0].value.timePrimitive.label[:4]), int(r.col_7['values'][0].value.timePrimitive.label[:4])])) if (len(r.col_6['values'])>0) and (len(r.col_7['values'])>0) else int(
        r.col_6['values'][0].value.timePrimitive.label[:4]) if (len(r.col_6['values'])>0) else int(r.col_7['values'][0].value.timePrimitive.label[:4]) if len(r.col_7['values'])>0 else np.nan]
  
  
    # Occours before
    element += ['-'.join(str(v.pk_entity) for v in r.col_8.entities) if len(r.col_8.entities)>0 else np.nan]
    
    # Occours after
    element += ['-'.join(str(v.pk_entity) for v in r.col_9.entities) if len(r.col_9.entities)>0 else np.nan]
    
    ll.append(element)

localisation = pd.DataFrame(ll[1:], columns=ll[0])
localisation.head()    


,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,1598.0,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1591.0,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN


> semplifica query in Geovistory: ridurre time-span

In [40]:
def localisation_generator(file):
    file = file
    with open(file, encoding='utf-8') as json_file:
        data_localisation = json.load(json_file)
    dmlocalisation = DotMap(data_localisation)
    dmlocalisation_rows = dmlocalisation.rows

    # Localisation list = ll
    ll = [['id_localisation', 'pk_person', 'person', 'geographical_place', 'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time', 'end', 'occours_before', 'occours_after']]

    for r in dmlocalisation_rows:
        element = []

        # id_localisatio, pk_person, person, geo_place + (id), 
        ## valuta la comodità di tenere separato l'ID geographical Place, geographical place, type, coordinate (x,y), localization type 
        element += [r.col_0.entity.pk_entity, r.col_1.entities[0].pk_entity, clean_up_str(r.col_1.entities[0].entity_label), (r.col_2.entities[0].entity_label + f' ({ r.col_2.entities[0].pk_entity})')  if len(
            r.col_2.entities)>0 else np.nan, r.col_10.entities[0].entity_label if len(r.col_10.entities)>=1 else np.nan]
        if len(r.col_12['values'])>0:    
            element += [r.col_12['values'][0].value.geometry.geoJSON.coordinates[1], r.col_12['values'][0].value.geometry.geoJSON.coordinates[0]]
        else:
            element +=[np.nan, np.nan]
        element += [r.col_11.entities[0].entity_label if len(r.col_11.entities)>0 else np.nan]

        # Begin
        element += [round(np.mean([int(r.col_3['values'][0].value.timePrimitive.label[:4]), int(r.col_4['values'][0].value.timePrimitive.label[:4])])) if (len(r.col_3['values'])>0) and (len(r.col_4['values'])>0) else int(
            r.col_3['values'][0].value.timePrimitive.label[:4]) if (len(r.col_3['values'])>0) else int(r.col_4['values'][0].value.timePrimitive.label[:4]) if len(r.col_4['values'])>0 else np.nan]
        
        # At some time within
        element += [int(r.col_5['values'][0].value.timePrimitive.label[:4]) if len(r.col_5['values'])>0 else np.nan]

        # End
        element += [round(np.mean([int(r.col_6['values'][0].value.timePrimitive.label[:4]), int(r.col_7['values'][0].value.timePrimitive.label[:4])])) if (len(r.col_6['values'])>0) and (len(r.col_7['values'])>0) else int(
            r.col_6['values'][0].value.timePrimitive.label[:4]) if (len(r.col_6['values'])>0) else int(r.col_7['values'][0].value.timePrimitive.label[:4]) if len(r.col_7['values'])>0 else np.nan]
    
    
        # Occours before
        element += ['-'.join(str(v.pk_entity) for v in r.col_8.entities) if len(r.col_8.entities)>0 else np.nan]
        ## MODIFICATO IN: element += [[ str(v.pk_entity) for v in r.col_8.entities] if len(r.col_8.entities)>0 else np.nan]
        
        # Occours after
        element += ['-'.join(str(v.pk_entity) for v in r.col_9.entities) if len(r.col_9.entities)>0 else np.nan]
        ## MODIFICATO IN:  element += [[str(v.pk_entity) for v in r.col_9.entities] if len(r.col_9.entities)>0 else np.nan]
        
        ll.append(element)

    return pd.DataFrame(ll[1:], columns=ll[0])

In [41]:
display(loc := localisation_generator('geovistory_query_exports/nb_loclaisation_20220512.json'))

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,1598.0,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1591.0,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN
14876,999714,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",S. Eufemia (742937),Parrocchia,45.426900,12.323600,Abitazione,NaN,1597.0,NaN,NaN,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN
14878,999734,999628,"Giacomo di Giulio Zambellini (da Bergamo, vend...",S. Geremia (742972),Parrocchia,45.442756,12.325453,Abitazione,NaN,1597.0,NaN,NaN,NaN


#### Esplorazione del dataset

In [42]:
loc[loc.occours_before.notnull() & loc.occours_after.notnull()].sort_values(by='id_localisation', ascending=False)

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
13471,2225343,2225323,"Pietro di Agostino Lin (grigione, vender vin)",Murano (768981),Luogo abitato,45.458333,12.353333,Luogo di lavoro,NaN,NaN,NaN,2225338,2225348
9092,1808475,1808447,Pietro di Paolo Cresci (anconetano),S. Polo (743160),Parrocchia,45.437056,12.329722,Abitazione,NaN,NaN,NaN,1808472,1808478
9091,1808472,1808447,Pietro di Paolo Cresci (anconetano),S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,NaN,NaN,1808467,1808475
4644,1462335,1461964,Maria di Leonardo (da Castelnovo del Friuli),S. Ternita (743244),Parrocchia,45.437257,12.348856,Abitazione,1588.0,NaN,NaN,1462312,1462349
4547,1341829,1341688,Giulia vedova di Giovanni Maria Fasolari,S. Fantin (742944),Parrocchia,45.433868,12.334100,Abitazione e possibile luogo di lavoro,NaN,NaN,NaN,1341835,1341824
1211,1084271,1084009,Giovanni di Giovanni Maria Salchi (da Piuro),Cannaregio (945713),Sestiere,45.446000,12.327000,Luogo di lavoro,NaN,NaN,NaN,1084296,1084261
1065,1076969,1076903,Sebastiano di Giovanni Angelo Demetri (dal Fri...,S. Moisè (743125),Parrocchia,45.433056,12.336111,Luogo di lavoro,1591.0,NaN,NaN,1076974,1076952
1012,1074686,1074671,Angela di Baldassarre (da Vicenza),S. Agostin (742860),Parrocchia,45.438717,12.328309,Abitazione e luogo di lavoro,NaN,NaN,NaN,1074681,1074691
725,1061131,1061101,Elena di Francesco (da Rineia),S. Maria Formosa (743074),Parrocchia,45.436900,12.341100,Abitazione,NaN,NaN,NaN,1061126,1061136
525,1051124,1050794,Orsola di Valentino Del Basso (da Trissino),S. Cassiano (742916),Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,NaN,1051114,1051120


In [43]:
loc[loc.occours_before.notnull() & loc.occours_before.str.contains('-')]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
516,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,1050961-1050934,NaN
524,1051120,1050794,Orsola di Valentino Del Basso (da Trissino),S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,1051124-1051114,NaN
4546,1341824,1341688,Giulia vedova di Giovanni Maria Fasolari,S. Aponal (742853),Parrocchia,45.437940,12.332610,Luogo di lavoro e possibile abitazione,NaN,NaN,NaN,1341829-1341835,NaN
4563,1342274,1341642,Lucrezia Sanudo (capodistriana),S. Aponal (742853),Parrocchia,45.437940,12.332610,Abitazione,NaN,NaN,NaN,1342279-1342283,NaN


In [44]:
loc[loc.occours_before.notnull()]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
91,1012100,1011744,"Antonio di Antonio Murla (da Fonzaso, samiter)",S. Marcuola (743046),Parrocchia,45.442892,12.328656,Luogo di lavoro,NaN,NaN,NaN,1011924,NaN
178,1032357,1032193,Domenico di Giorgio Negroni (marangon dell’ars...,S. Vio (743272),Parrocchia,45.430563,12.330241,Abitazione,NaN,NaN,NaN,1032352,NaN
187,1032496,1032116,Bradamante di Francesco Canano (da Pordenone) ...,S. Martin (743111),Parrocchia,45.434639,12.348964,Abitazione,NaN,NaN,NaN,1032485,NaN
356,1042187,1041703,"Giovanni di Bartolomeo Perotti (da Cancello, s...",S. Stin (743237),Parrocchia,45.437721,12.326848,Abitazione e luogo di lavoro,NaN,NaN,NaN,1042169,NaN
472,1049237,1049147,"Giovanni di Bartolomeo Targa (da Venezia, becher)",S. Margherita (743053),Parrocchia,45.435086,12.324154,Abitazione,NaN,NaN,NaN,1049197,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,985233,827944,Domenico di Nicolò (samiter),S. Geremia (742972),Parrocchia,45.442756,12.325453,Luogo di lavoro,NaN,NaN,NaN,827996,NaN
14704,986833,986767,"Domenico di Antonio Burattini (da Abano Terme,...",S. Polo (743160),Parrocchia,45.437056,12.329722,Luogo di lavoro,NaN,NaN,NaN,986782,NaN
14713,988166,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",S. Maria Zobenigo (743090),Parrocchia,45.432590,12.332840,Abitazione,1582.0,NaN,NaN,988161,NaN
14746,989448,989366,"Grazioso di Pietro Della Bona (da Bossico, mis...",S. Samuel (743181),Parrocchia,45.433611,12.328056,Abitazione,NaN,NaN,NaN,989443,NaN


In [45]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()


C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_19756/2156092610.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [46]:
loc['occours_before']= loc.occours_before.progress_apply(lambda x: x.split('-') if isinstance(x, str) else np.nan)
loc['occours_after'] = loc.occours_after.apply(lambda x: x.split('-') if type(x)==str else np.nan)

  0%|          | 0/14880 [00:00<?, ?it/s]

In [47]:
loc[loc.occours_before.notnull()]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
91,1012100,1011744,"Antonio di Antonio Murla (da Fonzaso, samiter)",S. Marcuola (743046),Parrocchia,45.442892,12.328656,Luogo di lavoro,NaN,NaN,NaN,[1011924],NaN
178,1032357,1032193,Domenico di Giorgio Negroni (marangon dell’ars...,S. Vio (743272),Parrocchia,45.430563,12.330241,Abitazione,NaN,NaN,NaN,[1032352],NaN
187,1032496,1032116,Bradamante di Francesco Canano (da Pordenone) ...,S. Martin (743111),Parrocchia,45.434639,12.348964,Abitazione,NaN,NaN,NaN,[1032485],NaN
356,1042187,1041703,"Giovanni di Bartolomeo Perotti (da Cancello, s...",S. Stin (743237),Parrocchia,45.437721,12.326848,Abitazione e luogo di lavoro,NaN,NaN,NaN,[1042169],NaN
472,1049237,1049147,"Giovanni di Bartolomeo Targa (da Venezia, becher)",S. Margherita (743053),Parrocchia,45.435086,12.324154,Abitazione,NaN,NaN,NaN,[1049197],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,985233,827944,Domenico di Nicolò (samiter),S. Geremia (742972),Parrocchia,45.442756,12.325453,Luogo di lavoro,NaN,NaN,NaN,[827996],NaN
14704,986833,986767,"Domenico di Antonio Burattini (da Abano Terme,...",S. Polo (743160),Parrocchia,45.437056,12.329722,Luogo di lavoro,NaN,NaN,NaN,[986782],NaN
14713,988166,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",S. Maria Zobenigo (743090),Parrocchia,45.432590,12.332840,Abitazione,1582.0,NaN,NaN,[988161],NaN
14746,989448,989366,"Grazioso di Pietro Della Bona (da Bossico, mis...",S. Samuel (743181),Parrocchia,45.433611,12.328056,Abitazione,NaN,NaN,NaN,[989443],NaN


In [48]:
loc[loc.occours_before.notnull() & loc.occours_before.apply(lambda x: type(x)==list)]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
91,1012100,1011744,"Antonio di Antonio Murla (da Fonzaso, samiter)",S. Marcuola (743046),Parrocchia,45.442892,12.328656,Luogo di lavoro,NaN,NaN,NaN,[1011924],NaN
178,1032357,1032193,Domenico di Giorgio Negroni (marangon dell’ars...,S. Vio (743272),Parrocchia,45.430563,12.330241,Abitazione,NaN,NaN,NaN,[1032352],NaN
187,1032496,1032116,Bradamante di Francesco Canano (da Pordenone) ...,S. Martin (743111),Parrocchia,45.434639,12.348964,Abitazione,NaN,NaN,NaN,[1032485],NaN
356,1042187,1041703,"Giovanni di Bartolomeo Perotti (da Cancello, s...",S. Stin (743237),Parrocchia,45.437721,12.326848,Abitazione e luogo di lavoro,NaN,NaN,NaN,[1042169],NaN
472,1049237,1049147,"Giovanni di Bartolomeo Targa (da Venezia, becher)",S. Margherita (743053),Parrocchia,45.435086,12.324154,Abitazione,NaN,NaN,NaN,[1049197],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14694,985233,827944,Domenico di Nicolò (samiter),S. Geremia (742972),Parrocchia,45.442756,12.325453,Luogo di lavoro,NaN,NaN,NaN,[827996],NaN
14704,986833,986767,"Domenico di Antonio Burattini (da Abano Terme,...",S. Polo (743160),Parrocchia,45.437056,12.329722,Luogo di lavoro,NaN,NaN,NaN,[986782],NaN
14713,988166,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",S. Maria Zobenigo (743090),Parrocchia,45.432590,12.332840,Abitazione,1582.0,NaN,NaN,[988161],NaN
14746,989448,989366,"Grazioso di Pietro Della Bona (da Bossico, mis...",S. Samuel (743181),Parrocchia,45.433611,12.328056,Abitazione,NaN,NaN,NaN,[989443],NaN


In [49]:
localisation  = list(loc.columns); localisation += ['sin_ob']; localisation = [localisation] 

for i, e in loc.iterrows():
    if type(e.occours_before) == list:
        for el in e.occours_before: 
            localisation.append(list(e) + [el])
    else:
        localisation.append(list(e) + [np.nan])
        
display(localisation := pd.DataFrame(localisation[1:], columns=localisation[0]))
        


,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,sin_ob
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,1598.0,NaN,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1591.0,NaN,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14879,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN,NaN
14880,999714,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",S. Eufemia (742937),Parrocchia,45.426900,12.323600,Abitazione,NaN,1597.0,NaN,NaN,NaN,NaN
14881,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN,NaN
14882,999734,999628,"Giacomo di Giulio Zambellini (da Bergamo, vend...",S. Geremia (742972),Parrocchia,45.442756,12.325453,Abitazione,NaN,1597.0,NaN,NaN,NaN,NaN


In [50]:
localisation[localisation.occours_before.notnull() & localisation.occours_before.apply(lambda x: len(x)>1 if type(x)==list else 0)]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,sin_ob
516,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,"[1050961, 1050934]",NaN,1050961
517,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,"[1050961, 1050934]",NaN,1050934
525,1051120,1050794,Orsola di Valentino Del Basso (da Trissino),S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,"[1051124, 1051114]",NaN,1051124
526,1051120,1050794,Orsola di Valentino Del Basso (da Trissino),S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,NaN,"[1051124, 1051114]",NaN,1051114
4548,1341824,1341688,Giulia vedova di Giovanni Maria Fasolari,S. Aponal (742853),Parrocchia,45.437940,12.332610,Luogo di lavoro e possibile abitazione,NaN,NaN,NaN,"[1341829, 1341835]",NaN,1341829
4549,1341824,1341688,Giulia vedova di Giovanni Maria Fasolari,S. Aponal (742853),Parrocchia,45.437940,12.332610,Luogo di lavoro e possibile abitazione,NaN,NaN,NaN,"[1341829, 1341835]",NaN,1341835
4566,1342274,1341642,Lucrezia Sanudo (capodistriana),S. Aponal (742853),Parrocchia,45.437940,12.332610,Abitazione,NaN,NaN,NaN,"[1342279, 1342283]",NaN,1342279
4567,1342274,1341642,Lucrezia Sanudo (capodistriana),S. Aponal (742853),Parrocchia,45.437940,12.332610,Abitazione,NaN,NaN,NaN,"[1342279, 1342283]",NaN,1342283


In [51]:
# localisation_integr = localisation[['id_localisation', 'pk_person', 'geographical_place',
#        'GP_type', 'lat', 'lon']]
# localisation_integr.rename(columns={'id_localisation':'sin_ob', 'pk_person':'_pkp', 'geographical_place':'_gp',
#        'GP_type':'_gpt', 'lat':'_lat', 'lon':'_lon'}, inplace=True)


In [52]:
localisation['id_localisation'] = localisation['id_localisation'].astype(str) 

In [53]:
localisation = localisation.merge(localisation, left_on='sin_ob', right_on='id_localisation', how='left', suffixes=(None, '_x'))

In [54]:
data = localisation[localisation.pk_person.isin(set([x for x in localisation[localisation.sin_ob.notnull() & localisation.sin_ob_x.notnull()].pk_person]))]

In [55]:
data[data.GP_type.str.contains('(?i)par') & data.GP_type_x.str.contains('(?i)par')]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
516,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,...,Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,NaN,[1050934],[1050948],1050934
517,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,...,Parrocchia,45.435086,12.324154,Abitazione,NaN,1598.0,NaN,NaN,"[1050948, 1050961]",NaN
518,1050961,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Cassiano (742916),Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,...,Parrocchia,45.435086,12.324154,Abitazione,NaN,1598.0,NaN,NaN,"[1050948, 1050961]",NaN
525,1051120,1050794,Orsola di Valentino Del Basso (da Trissino),S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,...,Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,NaN,[1051114],[1051120],1051114
526,1051120,1050794,Orsola di Valentino Del Basso (da Trissino),S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,...,Parrocchia,45.436900,12.341100,Abitazione,NaN,1598.0,NaN,NaN,"[1051120, 1051124]",NaN
527,1051124,1050794,Orsola di Valentino Del Basso (da Trissino),S. Cassiano (742916),Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,...,Parrocchia,45.436900,12.341100,Abitazione,NaN,1598.0,NaN,NaN,"[1051120, 1051124]",NaN
727,1061131,1061101,Elena di Francesco (da Rineia),S. Maria Formosa (743074),Parrocchia,45.436900,12.341100,Abitazione,NaN,NaN,...,Parrocchia,45.438120,12.346078,Abitazione,NaN,1592.0,NaN,NaN,[1061131],NaN
728,1061136,1061101,Elena di Francesco (da Rineia),S. Martin (743111),Parrocchia,45.434639,12.348964,Abitazione,NaN,1588.0,...,Parrocchia,45.436900,12.341100,Abitazione,NaN,NaN,NaN,[1061126],[1061136],1061126
1014,1074686,1074671,Angela di Baldassarre (da Vicenza),S. Agostin (742860),Parrocchia,45.438717,12.328309,Abitazione e luogo di lavoro,NaN,NaN,...,Parrocchia,45.438927,12.320719,Abitazione e luogo di lavoro,NaN,1594.0,NaN,NaN,[1074686],NaN
1015,1074691,1074671,Angela di Baldassarre (da Vicenza),S. Polo (743160),Parrocchia,45.437056,12.329722,Abitazione e luogo di lavoro,1578.0,NaN,...,Parrocchia,45.438717,12.328309,Abitazione e luogo di lavoro,NaN,NaN,NaN,[1074681],[1074691],1074681


In [56]:
import keplergl

In [57]:
config = {'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'r3dzv5g', 'type': 'arc', 'config': {'dataId': 'data', 'label': 'Point', 'color': [179, 173, 158], 'highlightColor': [252, 242, 26, 255], 'columns': {'lat0': 'lat', 'lng0': 'lon', 'lat1': 'lat_x', 'lng1': 'lon_x'}, 'isVisible': True, 'visConfig': {'opacity': 0.8, 'thickness': 2, 'colorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#5A1846', '#900C3F', '#C70039', '#E3611C', '#F1920E', '#FFC300']}, 'sizeRange': [0, 10], 'targetColor': None}, 'hidden': False, 'textLabel': [{'field': None, 'color': [255, 255, 255], 'size': 18, 'offset': [0, 0], 'anchor': 'start', 'alignment': 'center'}]}, 'visualChannels': {'colorField': {'name': 'pk_person', 'type': 'integer'}, 'colorScale': 'quantile', 'sizeField': None, 'sizeScale': 'linear'}}], 'interactionConfig': {'tooltip': {'fieldsToShow': {'data': [{'name': 'person', 'format': None}, {'name': 'GP_type', 'format': None}, {'name': 'geographical_place_x', 'format': None}, {'name': 'geographical_place', 'format': None}]}, 'compareMode': False, 'compareType': 'absolute', 'enabled': True}, 'brush': {'size': 0.5, 'enabled': False}, 'geocoder': {'enabled': False}, 'coordinate': {'enabled': False}}, 'layerBlending': 'normal', 'splitMaps': [], 'animationConfig': {'currentTime': None, 'speed': 1}}, 'mapState': {'bearing': 24, 'dragRotate': True, 'latitude': 45.43632793902099, 'longitude': 12.338365338468364, 'pitch': 50, 'zoom': 14.239326948261827, 'isSplit': False}, 'mapStyle': {'styleType': 'muted', 'topLayerGroups': {}, 'visibleLayerGroups': {'label': True, 'road': True, 'border': False, 'building': True, 'water': True, 'land': True, '3d building': True}, 'threeDBuildingColor': [224.4071295378559, 224.4071295378559, 224.4071295378559], 'mapStyles': {}}}}

In [58]:
mappa = keplergl.KeplerGl(height=600, data={"data": data}, config = config)
mappa

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


c:\Users\Jacopo COSSU\anaconda_surrogato\v_env_surog\lib\site-packages\jupyter_client\session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'r3dzv5g', 'type': …

In [59]:
localisation[localisation.pk_person.isin([x.pk_person for i, x in loc[loc.GP_type.notnull() & loc.GP_type.str.contains('(?i)par')].groupby('pk_person').agg(
    {'id_localisation':'count'}).sort_values(by='id_localisation', ascending=False).reset_index().iterrows() if x.id_localisation>=3])]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
141,1026799,1026703,"Paolo di Bartolomeo Veraldi (romano, cantor)",S. Moisè (743125),Parrocchia,45.433056,12.336111,Abitazione,NaN,1599.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
207,1034666,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",S. Maria Mater Domini (743060),Parrocchia,45.439722,12.330278,Abitazione,NaN,1599.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,1034715,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1584.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
515,1050934,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Margherita (743053),Parrocchia,45.435086,12.324154,Abitazione,NaN,1598.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
516,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,NaN,NaN,...,Parrocchia,45.439701,12.331932,Abitazione,NaN,NaN,NaN,[1050934],[1050948],1050934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13509,2226703,2226640,Marino di Simone Dal Pesce (da Condino),S. Cassiano (742916),Parrocchia,45.439701,12.331932,Abitazione,NaN,1639.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,988128,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1582.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,988161,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",S. Stin (743237),Parrocchia,45.437721,12.326848,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14717,988166,988086,"Bartolmeo di Domenico Bernacioli (masser, da M...",S. Maria Zobenigo (743090),Parrocchia,45.432590,12.332840,Abitazione,1582.0,NaN,...,Parrocchia,45.437721,12.326848,Abitazione,NaN,1597.0,NaN,NaN,[988166],NaN


In [60]:
localisation

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,1598.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1591.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14879,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14880,999714,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",S. Eufemia (742937),Parrocchia,45.426900,12.323600,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14881,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14882,999734,999628,"Giacomo di Giulio Zambellini (da Bergamo, vend...",S. Geremia (742972),Parrocchia,45.442756,12.325453,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
localisation.columns

Index(['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob', 'id_localisation_x',
       'pk_person_x', 'person_x', 'geographical_place_x', 'GP_type_x', 'lat_x',
       'lon_x', 'localisation_type_x', 'begin_x', 'at_some_time_x', 'end_x',
       'occours_before_x', 'occours_after_x', 'sin_ob_x'],
      dtype='object')

In [62]:
len(localisation[localisation.geographical_place.notnull() & localisation.geographical_place.str.contains('Venezia IT')  & localisation.begin.notnull() & localisation.end.isnull() & (localisation.occours_before.notnull() | localisation.occours_after.notnull()) & ((localisation.occours_before.notnull() & localisation.occours_after.notnull()) ==False)][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after']]
)

21

In [63]:
localisation[localisation.geographical_place.notnull() & localisation.geographical_place.str.contains('Venezia IT')  & localisation.begin.notnull() & localisation.end.isnull() & (localisation.occours_before.notnull() | localisation.occours_after.notnull()) & ((localisation.occours_before.notnull() & localisation.occours_after.notnull()) ==False)][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after']]


,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
2036,1123094,1123073,Giacomo di Aloisio Aprilis (da Cusano),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1584.0,NaN,NaN,NaN,[1123170]
2134,1127918,1127892,Sebastiano Giovanni di Francesco Giudici (da G...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,NaN,[1127958]
2148,1128368,1128343,"Paolo di Rocco De Rubertis (bresciano, murer)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1584.0,NaN,NaN,NaN,[1298688]
2175,1129294,1129275,"Giovanni di Antonio (da Chiavenna, tesser)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1584.0,NaN,NaN,NaN,[1129329]
3363,1249192,1249167,"Antonio di Giacomo Varino (bergamasco, portado...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1584.0,NaN,NaN,NaN,[1249229]
4247,1305069,1305044,"Sgualdo di Andrea Belfini (cadorino, marangon)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1582.0,NaN,NaN,NaN,[1305107]
4249,1305107,1305082,Battista di Giovanni Maria Ambrogi (dalla Val ...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,[1305069],NaN
4318,1320545,1320515,Lucia di Tommaso Viaro (da Loria) moglie di Gi...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1582.0,NaN,NaN,NaN,[1320503]
4527,1340857,1340835,Giovanni Battista di Lazzaro Mazzoleni (da Arb...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,[1340918]
4804,1473728,1473708,Bernardino di Giacomo Di Bortolo (da San Giorg...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,[1473761],NaN


In [64]:
tag = [['id_localisation', 'tag', 'note']]
processetti = processetti_generator('geovistory_query_exports/nb_overwiev_processetti_20220510.json')
testimoni = persons_generator('geovistory_query_exports/nb_testimoni_20220510.json', 'testimoni')

"""
abitazione_venezia --> rappresenta una generica indicazione di presenza del soggetto a Venezia (concettualmente equivale al luogo d'abitazione)
arrivo_venezia --> Indica l'arrivo a Venezia di un soggetto
soggiorno_venezia --> Indica una permanenza a Venezia limitata nel tempo
"""

for person in localisation.pk_person.unique():
   df_person = localisation[localisation.pk_person == person]
   sintesi_person = df_person.groupby(['geographical_place']).agg({'id_localisation':'count'}).reset_index()
   # Arrivo a Venezia
   ## Nuomero di menzioni per persona
   if 'Venezia IT (209678)' in sintesi_person.geographical_place.unique():
      mention = int(sintesi_person[sintesi_person.geographical_place == 'Venezia IT (209678)'].id_localisation)
   else:
      mention = 0
   sub_set = localisation[(localisation.pk_person == person ) & (localisation.geographical_place=='Venezia IT (209678)')].fillna(0)
## Se Venezia menzionata una sola volta
   if mention == 1:
      single_tag = [] 
      for i, e in sub_set.iterrows():
   ### Arco aperto
         if (e.begin!=0) and (e.end==0):
      #### Solo Begin
            if (e.occours_before==0) and (e.occours_after==0):
               single_tag += [e.id_localisation, 'arrivo_venezia', np.nan]       
      #### Begin & ((Occours_before) if pk_person != pk_person_x)
            elif  (e.occours_before!=0) or (e.occours_after!=0):
               if e.occours_before!=0:
                  if e.pk_person != e.pk_person_x:
                     # print(f'{e.id_localisation}/{e.pk_person}:{e.geographical_place}({e.GP_type})-->{e.geographical_place_x}({e.GP_type_x})') 
                     single_tag += [e.id_localisation, 'arrivo_venezia', np.nan]
                  else:
                     if e.GP_type_x in [ 'Città', 'Area geografica', 'Luogo abitato']:
                        single_tag += [e.id_localisation, 'soggiorno_venezia', 'Verifica consecutio'] # rappresenta nei fatti un arco chiuso --> 
                        # TODO: riprendi compilazione end
                     else:
                        # print(f'{e.id_localisation}/{e.pk_person}:{e.geographical_place}({e.GP_type})-->{e.geographical_place_x}({e.GP_type_x})')
                        single_tag += [e.id_localisation, 'arrivo_venezia', np.nan]

                     # print(f'{e.id_localisation}/{e.pk_person}:{e.geographical_place}({e.GP_type})-->{e.geographical_place_x}({e.GP_type_x})')
               elif e.occours_after!=0: # rappresenta nei fatti un arco aperto
                  single_tag += [e.id_localisation, 'arrivo_venezia', 'Verifica occours after']

                  """
                  if e.pk_person != localisation.loc[localisation.id_localisation==e.occours_after].pk_person.values[0]:
                     print(f'{e.id_localisation}/{e.pk_person}:{e.geographical_place}({e.GP_type})<--
                     {(localisation.loc[localisation.id_localisation==e.occours_after].geographical_place_x.values[0])}({localisation.loc[localisation.id_localisation==e.occours_after].GP_type_x.values[0]}): {localisation.loc[localisation.id_localisation==e.occours_after].pk_person.values[0]}')
                  else:
                     print(f'{e.id_localisation}/{e.pk_person}:{e.geographical_place}({e.GP_type})<--
                     {(localisation.loc[localisation.id_localisation==e.occours_after].geographical_place_x.values[0])}({localisation.loc[localisation.id_localisation==e.occours_after].GP_type_x.values[0]}): {localisation.loc[localisation.id_localisation==e.occours_after].pk_person.values[0]}')
                  """

   #### Solo at_some_time
         elif e.at_some_time !=0:
            cross_reference = (processetti[processetti.pk_person == e.pk_person].anno.values[0] if len(processetti[processetti.pk_person == e.pk_person])>0 else processetti[processetti.pk_processetto == (testimoni[testimoni.pk_person == e.pk_person].pk_processetto.values[0])].anno.values[0])
            # print(f' {e.pk_person} --> {e.at_some_time} = {processetti[processetti.pk_person == e.pk_person].anno.values[0] if len(processetti[processetti.pk_person == e.pk_person])>0 else processetti[processetti.pk_processetto == (testimoni[testimoni.pk_person == e.pk_person].pk_processetto.values[0])].anno.values[0]}')
            if (e.occours_before==0) and (e.occours_after==0): 
               if e.at_some_time < cross_reference:
                  # TODO: da verificare, possibili errori -- probabilmente da intendersi come Begin 
                  single_tag += [e.id_localisation, 'arrivo_venezia', 'verifica datazione (ast-->b)']
               elif e.at_some_time == cross_reference:
                  single_tag += [e.id_localisation, 'abitazione_venezia', 'verifica datazione (ast-->b)']
               else: 
                  ## Costituisce eventualmente un errore o, al limite, la presenza di più processetti di riferimento
                  single_tag += [e.id_localisation, 'arrivo_venezia', 'Errore - verifica datazione']
            elif (e.occours_before!=0) or (e.occours_after!=0):
               if e.occours_before!=0:
                  if e.pk_person == e.pk_person_x:
                     single_tag += [e.id_localisation, 'soggiorno_venezia', 'Verifica consecutio']
                  else:
                     if e.at_some_time < cross_reference:      
                        single_tag += [e.id_localisation, 'arrivo_venezia', 'verifica datazione (ast-->b)']
                     elif e.at_some_time == cross_reference:
                        single_tag += [e.id_localisation, 'abitazione_venezia', 'verifica datazione (ast-->b)']
                     else:
                        single_tag += [e.id_localisation, 'arrivo_venezia', 'Errore - verifica datazione']
               elif e.occours_after!=0: 
                  if e.at_some_time == cross_reference:
                     single_tag += [e.id_localisation, 'abitazione_venezia', 'verifica datazione (ast-->b)']
                  else:   
                     single_tag += [e.id_localisation, 'arrivo_venezia', 'verifica datazione (ast-->b)']
            else: ##((e.occours_before!=0) & (e.occours_after!=0))
               single_tag += [e.id_localisation, 'soggiorno_venezia', 'Verifica consecutio']

            # print(f' {e.pk_person} = {e.at_some_time} <--> {processetti[processetti.pk_person == e.pk_person].anno.values[0] if len(processetti[processetti.pk_person == e.pk_person])>0 else processetti[processetti.pk_processetto == (testimoni[testimoni.pk_person == e.pk_person].pk_processetto.values[0])].anno.values[0]}')

   ### Arco chiuso
      #### Begin & End
         elif (e.begin!=0) and (e.end!=0):
            if (e.occours_before==0) and (e.occours_after==0):
               if e.end <= (processetti[processetti.pk_person == e.pk_person].anno.values[0] if len(processetti[processetti.pk_person == e.pk_person])>0 else processetti[processetti.pk_processetto == (testimoni[testimoni.pk_person == e.pk_person].pk_processetto.values[0])].anno.values[0]):
                  single_tag += [e.id_localisation, 'soggiorno_venezia', np.nan]
               else:
                 single_tag += [e.id_localisation, 'soggiorno_venezia', 'Errore - verifica datazione']
            elif (e.occours_before!=0) or (e.occours_after!=0):
               single_tag += [e.id_localisation, 'soggiorno_venezia', np.nan]
      #### Solo indicazione qualitativa
         elif (e.begin==0) and (e.end==0):
            if (e.occours_before!=0) and (e.occours_after!=0):
               single_tag += [e.id_localisation, 'soggiorno_venezia', np.nan]
            elif (e.occours_before!=0) or (e.occours_after!=0):
               single_tag += [e.id_localisation, 'arrivo_venezia', np.nan]
               # TODO: recupera datazione dal secondo event
      #### Assenza di riferimento       
            else:
               single_tag += [e.id_localisation, 'verifica_venezia', 'Errore - datazione assente']
               # TODO: recupera datazione da processetto
               
               #print(f'{e.pk_person}  -- {processetti[processetti.pk_person == e.pk_person].anno.values[0] if len(processetti[processetti.pk_person == e.pk_person])>0 else processetti[processetti.pk_processetto == (testimoni[testimoni.pk_person == e.pk_person].pk_processetto.values[0])].anno.values[0]}')     
         else:
            single_tag += [e.id_localisation, 'soggiorno_venezia', np.nan]

         tag.append(single_tag)
   elif mention>1:
      provisory_tag = []
      single_tag = []
           
      for i,e in sub_set.iterrows():
         if (e.begin !=0) and (e.end!=0):
            provisory_tag += ['soggiorno_venezia']
         elif (e.begin !=0) and (e.end==0):
            if (e.occours_before == 0):
               provisory_tag += ['arrivo_venezia']
            else:
               if e.pk_person != e.pk_person_x:
                  provisory_tag += ['arrivo_venezia']
               else:
                  provisory_tag += ['soggiorno_venezia']

         elif (e.begin ==0) and (e.end!=0):
            provisory_tag += ['soggiorno_venezia']
         elif e.at_some_time!=0:
            if (e.occours_before == 0):
               provisory_tag += ['abitazione_venezia']
            else:
               provisory_tag += ['soggiorno_venezia']
         else:
            provisory_tag += ['altro_venezia']
            print(e)

      sub_set['provisory_tag'] = provisory_tag
      if len(sub_set[sub_set.provisory_tag=='arrivo_venezia'])<=1:
         for i,e in sub_set.iterrows():
            tag.append([e.id_localisation, e.provisory_tag, np.nan]) 
      else:
         counter = 0
         for i,e in sub_set.sort_values(ascending=True, by='begin').iterrows():
            if e.provisory_tag == 'arrivo_venezia':
               if counter == 0:
                  single_tag = [e.id_localisation, e.provisory_tag, np.nan]
                  counter += 1
               else:
                  single_tag = [e.id_localisation, 'abitazione_venezia', 'Errore - unificare'] 
            else:
               single_tag = [e.id_localisation, e.provisory_tag, np.nan]
              
            tag.append(single_tag)
            
print(len(localisation) == len(arrivo_venezia := pd.DataFrame(tag[1:], columns=tag[0])))


False


In [65]:
loc['id_localisation'] = loc.id_localisation.astype(str)
print(type(loc.id_localisation[0]), type(arrivo_venezia.id_localisation[0]))

<class 'str'> <class 'str'>


In [66]:
loca = loc.merge(arrivo_venezia, how='left', on='id_localisation')
loca = loca[loca.tag.notnull()]; loca

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,tag,note
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1591.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
5,1006553,1006430,Pietro di Giovanni Maria (da Venezia),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1547.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
7,1006588,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
10,1006634,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1589.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14871,999021,998887,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14873,999197,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN


In [67]:
# 1599 - 1603
from statistics import mean


b = 1599
e = 1603
x = set(range(b, e, 1))

ba = 1602
fa= 1607
y = set(range(ba, fa, 1))

print([mean(x), mean(y)]) 



[1600.5, 1604]


In [68]:
loca.note.unique()

array([nan, 'verifica datazione (ast-->b)', 'Errore - datazione assente',
       'Errore - unificare', 'Verifica occours after',
       'Verifica consecutio'], dtype=object)

In [69]:
localisation.columns

Index(['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob', 'id_localisation_x',
       'pk_person_x', 'person_x', 'geographical_place_x', 'GP_type_x', 'lat_x',
       'lon_x', 'localisation_type_x', 'begin_x', 'at_some_time_x', 'end_x',
       'occours_before_x', 'occours_after_x', 'sin_ob_x'],
      dtype='object')

In [70]:
localisation.pk_person.unique()

array([1006422,  999827, 1006430, ...,  999563,  999580,  999628],
      dtype=int64)

In [71]:
localisation.columns

Index(['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob', 'id_localisation_x',
       'pk_person_x', 'person_x', 'geographical_place_x', 'GP_type_x', 'lat_x',
       'lon_x', 'localisation_type_x', 'begin_x', 'at_some_time_x', 'end_x',
       'occours_before_x', 'occours_after_x', 'sin_ob_x'],
      dtype='object')

In [72]:
verifica = []
for e in localisation[localisation.begin.isnull() & localisation.end.notnull()].pk_person:
       verifica.append(e)

localisation[localisation.pk_person.isin(verifica)][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob']].sort_values(by='pk_person')

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,sin_ob
14260,774539,774507,Giovanni Francesco di Delaidi Gosetti (da Pesc...,S. Trovaso (743258),Parrocchia,45.430790,12.326010,Abitazione,NaN,NaN,1593.0,NaN,NaN,NaN
131,1026514,1012979,Caterina Albrizzi vedova di Francesco Bettinel...,S. Luca (743011),Parrocchia,45.435760,12.333780,Abitazione,NaN,1599.0,NaN,NaN,NaN,NaN
132,1026519,1012979,Caterina Albrizzi vedova di Francesco Bettinel...,S. Aponal (742853),Parrocchia,45.437940,12.332610,Abitazione,NaN,NaN,1591.0,NaN,NaN,NaN
1295,1088140,1034490,Sebastiano di Benedetto Zardino (da Ampezzo di...,Rialto (1065183),Area urbana,45.438008,12.335639,Luogo di lavoro,NaN,NaN,1596.0,NaN,NaN,NaN
1294,1088135,1034490,Sebastiano di Benedetto Zardino (da Ampezzo di...,S. Giovanni Grisostomo (743307),Parrocchia,45.439160,12.337260,Luogo di lavoro,NaN,1596.0,NaN,NaN,NaN,NaN
206,1034523,1034490,Sebastiano di Benedetto Zardino (da Ampezzo di...,S. Giovanni Grisostomo (743307),Parrocchia,45.439160,12.337260,Abitazione,NaN,1599.0,NaN,NaN,NaN,NaN
208,1034715,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1584.0,NaN,NaN,NaN,NaN,NaN
207,1034666,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",S. Maria Mater Domini (743060),Parrocchia,45.439722,12.330278,Abitazione,NaN,1599.0,NaN,NaN,NaN,NaN
1299,1088247,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",S. Aponal (742853),Parrocchia,45.437940,12.332610,Abitazione,NaN,1598.0,NaN,NaN,NaN,NaN
1300,1088254,1034505,"Giacomo di Pietro Dadia (da Ampezzo di Cadore,...",S. Giovanni Grisostomo (743307),Parrocchia,45.439160,12.337260,Luogo di lavoro,NaN,1596.0,NaN,NaN,NaN,NaN


In [75]:
oa  = [] 
for x in localisation.loc[localisation.begin.isnull() & localisation.at_some_time.isnull() & localisation.end.isnull() & localisation.occours_before.isnull() & localisation.occours_after.notnull()]['occours_after']:
    
    if len(x)<9:

        oa.append(x)



In [76]:
processetti

,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),1006422.0,Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",999599.0,Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",1006794.0,Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),1006856.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),1007216.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,EM 05,"Processetto di Gaspare Forzanini (da Bassano, ...",1,998606,"Gaspare Forzanini (da Bassano, calegher)",998390.0,Stato libero di “foresto”,1598-12-18,1598-12-18,1598.0,NaN,0
3793,EM 04,Processetto di Cristina vedova di Nicola De Ga...,1,998615,Cristina vedova di Nicola De Gaspari (da Venezia),998555.0,Vedovanza,1597-05-20,1597-05-20,1597.0,998630,1
3794,EM 04,Processetto di Giovanni Maria di Antonio (da L...,1,998913,"Giovanni Maria di Antonio (da Lovere, tesser d...",998798.0,Vedovanza,1597-05-21,1597-05-21,1597.0,998922,1
3795,EM 05,Processetto di Andrea di Angelo Pasetto (da Ma...,1,999218,"Andrea di Angelo Pasetto (da Magno, erbariol)",998838.0,Stato libero di “foresto”,1598-12-21,1598-12-21,1598.0,NaN,0


In [77]:
localisation.GP_type.unique()

array(['Parrocchia', 'Città', 'Sestiere', 'Area geografica',
       'Luogo abitato', 'Area urbana ', nan, 'Diocesi'], dtype=object)

In [78]:
localisation[localisation.id_localisation.apply(lambda x: x in oa)][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob']]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,sin_ob


In [79]:
modellino = pd.DataFrame([[1,2,3], [2,3,4]], columns=['a','b','c'])
modellino.loc[0]['a'] = 5
modellino

,a,b,c
0,5,2,3
1,2,3,4


In [80]:
id_local = '1880479'

setloc = localisation[localisation.id_localisation == id_local]
for i, e in setloc.iterrows():
    if not np.isnan(e.at_some_time):
        print(e.at_some_time, type(e.at_some_time), i)
        localisation['begin'][i], localisation['end'][i] = e.at_some_time, e.at_some_time

localisation[localisation.id_localisation == id_local].end

1595.0 <class 'float'> 9865


C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_19756/284240160.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisation['begin'][i], localisation['end'][i] = e.at_some_time, e.at_some_time


9865    1595.0
Name: end, dtype: float64

In [81]:
type(localisation['occours_after'][86])


list

In [82]:
localisation['occours_after'] = localisation.occours_after.apply(lambda x: x.split('-') if type(x)==str else np.nan)

In [83]:
localisation[['occours_after']]

,occours_after
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
14879,NaN
14880,NaN
14881,NaN
14882,NaN


In [84]:
type(arrivo_venezia['id_localisation'][0])

str

In [85]:
localisation['id_localisation'] = localisation['id_localisation'].astype(str)

arrivo_venezia = arrivo_venezia[['id_localisation', 'tag']].merge(localisation[['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end']], on='id_localisation', how='left')

arrivo_venezia[arrivo_venezia.begin.isnull()]

,id_localisation,tag,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end
115,1039734,arrivo_venezia,1039703,Margherita di Baldo Balzar (da Cerete Basso),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,1621.0,NaN
116,1039802,verifica_venezia,1039757,"Lorenzo pescador di Donato (da Cerete Basso, f...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN
117,1039831,verifica_venezia,1039811,"Bernardo di Morello Morelli (da Cerete Basso, ...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN
133,1041043,verifica_venezia,1040263,"Giuseppe di Giovanni Maria (da Venezia, forner)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN
134,1041061,verifica_venezia,1040276,Luca di Pietro Valerio (da Firenze),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5144,928524,abitazione_venezia,779721,"Simone di Caciuch (da Angora, mercante di zamb...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione e luogo di lavoro,NaN,1592.0,NaN
5160,942380,abitazione_venezia,942203,Gregorio di Luca Muschi(da Gemona del Friuli),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,1597.0,NaN
5161,946073,arrivo_venezia,945946,"Giacomo di Battista Commoto (speziale, da Zand...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,1587.0,NaN
5162,946102,arrivo_venezia,945978,"Francesco di Donato Pisetti (cerer, da Zandobb...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,1587.0,NaN


In [86]:
if '999739' in arrivo_venezia.id_localisation.values:
    print('ok')

ok


In [87]:
arrivo_venezia.tag.values

array(['arrivo_venezia', 'arrivo_venezia', 'arrivo_venezia', ...,
       'arrivo_venezia', 'arrivo_venezia', 'arrivo_venezia'], dtype=object)

In [97]:
display(localisation := localisation_generator('geovistory_query_exports/nb_loclaisation_20220512.json'))

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),Parrocchia,45.443800,12.332900,Abitazione,NaN,1598.0,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1591.0,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN
14876,999714,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",S. Eufemia (742937),Parrocchia,45.426900,12.323600,Abitazione,NaN,1597.0,NaN,NaN,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1580.0,NaN,NaN,NaN,NaN
14878,999734,999628,"Giacomo di Giulio Zambellini (da Bergamo, vend...",S. Geremia (742972),Parrocchia,45.442756,12.325453,Abitazione,NaN,1597.0,NaN,NaN,NaN


In [98]:
localisations  = list(localisation.columns); localisations += ['sin_ob']; localisations = [localisations] 

for i, e in localisation.iterrows():
    if type(e.occours_before) == list:
        for el in e.occours_before: 
            localisations.append(list(e) + [el])
    else:
        localisations.append(list(e) + [np.nan])
        
display((localisation := pd.DataFrame(localisations[1:], columns=localisations[0])).head(1))
        

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,sin_ob
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,NaN,NaN,NaN,NaN


In [99]:
localisation['id_localisation']=localisation['id_localisation'].astype(str)

In [100]:

localisation['occours_after'] = localisation.occours_after.apply(lambda x: x.split('-') if type(x)==str else np.nan)

arrivo_venezia = arrivo_venezia[arrivo_venezia.tag == 'arrivo_venezia'][['id_localisation', 'tag']].merge(localisation[['id_localisation', 'pk_person', 'person', 'geographical_place','GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time','end']], on='id_localisation', how='left')

import functools
dizionario_person_loc ={}

def compare_begins(x, y):
    try:
        return localisation[localisation.id_localisation == x].begin.values[0] - localisation[localisation.id_localisation == y].begin.values[0]
    except:
        print(x, y)

def get_el_by_id(id_local):
    for e in localisation[localisation.id_localisation == id_local].iterrows():
        return e

def get_max_year(array):
    max = 0
    for year in array:
        if max < year:
            max = year
    return max

def get_min_year(array):
    min = 2000.0
    for year in array:
        if min > year:
            min = year
    return min

def get_arrivo_venezia(e):
    return arrivo_venezia[arrivo_venezia.pk_person==e.pk_person].begin.values[0]

def fai_il_miracolo_per_begin(e):
    if e.GP_type in ['Parrocchia',  'Sestiere', 'Area urbana '] and e.pk_person in arrivo_venezia.pk_person.values:
        return get_arrivo_venezia(e)
    else:
        return 0
        # TODO: generatore_person --> person--> anno di nascita, attribuisci 0 come valore di default 

def  data_processetto(e):
    return 2000.0
    try:
        processetti
    except:
        processetti = processetti_generator('geovistory_query_exports/nb_overwiev_processetti_20220510.json')
    return processetti[processetti.pk_person == e.pk_person].anno.values[0]


     

def get_undefined_begin(e):
    if type(e.occours_after)==list:
        oa_end = []
        for id in e.occours_after:
            for i, el in localisation[localisation.id_localisation == id].iterrows():
                if not np.isnan(el.end):
                    oa_end.append(el.end)
                else:
                    set_begin_and_end(el.id_localisation)
                    oa_end.append(get_el_by_id(el.id_localisation).end)
        return get_max_year(oa_end)
    return fai_il_miracolo_per_begin(e)
    
        
def get_undefined_end(e):
    if type(e.occours_before)==list:
        oa_begin = []
        for id in e.occours_before:
            for i, el in localisation[localisation.id_localisation == id].iterrows():
                if not np.isnan(el.begin):
                    oa_begin.append(el.begin)
                else:
                    set_begin_and_end(el.id_localisation)
                    oa_begin.append(get_el_by_id(el.id_localisation).begin)
        return get_min_year(oa_begin)
    return data_processetto(e)

def set_begin_and_end(id_local):
    setloc = localisation[localisation.id_localisation == id_local]
    for i, e in setloc.iterrows():
        if not np.isnan(e.at_some_time):
            localisation['begin'][i], localisation['end'][i] = e.at_some_time, e.at_some_time
        else:
            if np.isnan(e.begin):
                localisation['begin'][i] = get_undefined_begin(e)
            if np.isnan(e.end):
                localisation['end'][i] = get_undefined_end(e)

def sort_localisations(localisations_per_person):
    
    for el in localisations_per_person:
        set_begin_and_end(el)
    return sorted(localisations_per_person, key= functools.cmp_to_key(compare_begins))

for person in localisation.pk_person.unique():
    dizionario_person_loc[person] = []
    inside = {}
    subset_person = localisation[localisation.pk_person == person]
    
    for i, r in subset_person.iterrows():
        dizionario_person_loc[person].append(r.id_localisation)
        if (type(r.sin_ob) != float ):
            if r.sin_ob not in dizionario_person_loc[person]:
                dizionario_person_loc[person].append(r.sin_ob)
            if type(r.occours_after)!= float:
                for el in r.occours_after:
                    if el not in dizionario_person_loc[person]:
                        dizionario_person_loc[person].append([el])
    try:
        dizionario_person_loc[person] = sort_localisations(dizionario_person_loc[person]) 
    except:
        print(dizionario_person_loc[person])   

   




C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_19756/658099061.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisation['begin'][i], localisation['end'][i] = e.at_some_time, e.at_some_time
C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_19756/658099061.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  localisation['end'][i] = get_undefined_end(e)
C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_19756/658099061.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

['1061126', '1061131', '1061136', '1061142']
['1808467', '1808472', '1808475', '1808478']
['2225338', '2225343', '2225348', '2225365']
['764982', '942564']


In [92]:
subset_person.columns

Index(['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after'],
      dtype='object')

In [ ]:
localisation[localisation.pk_person==1050827][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after']]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
515,1050934,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Margherita (743053),Parrocchia,45.435086,12.324154,Abitazione,1598.0,1598.0,1598.0,NaN,NaN
516,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,1600.0,NaN,1598.0,"[1050961, 1050934]",NaN
517,1050948,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,0.0,NaN,1598.0,"[1050961, 1050934]",NaN
518,1050961,1050827,"Giacomo di Pietro Legna (da Venezia, sanser)",S. Cassiano (742916),Parrocchia,45.439701,12.331932,Abitazione,1600.0,NaN,1598.0,[1050934],NaN


In [ ]:
dizionario_person_loc

{1006422: ['1006478', '1006461'],
 999827: ['1006473', '1006467', '1065195'],
 1006430: ['1006553', '1006544'],
 1006438: ['1006588', '1006583'],
 1006449: ['1006614'],
 1002990: ['1006634', '1006626'],
 999599: ['1006718', '1006699'],
 1006794: ['1006877', '1006871'],
 1006806: ['1006915', '1006910'],
 1006821: ['1006945', '1006940'],
 1006982: ['1007048'],
 1007018: ['1007116'],
 1006856: ['1007173', '1007142'],
 1007235: ['1007283', '1007278'],
 1007255: ['1007317', '1007312'],
 1007216: ['1007350', '1007345'],
 1007408: ['1007457', '1007441'],
 1007423: ['1007501'],
 1007388: ['1007552'],
 1007492: ['1007664'],
 1007877: ['1007922', '1007917'],
 1007833: ['1007940', '1007927'],
 1007901: ['1007993', '1007965', '1780212'],
 1007886: ['1007987'],
 1007842: ['1008050', '1008045'],
 1007818: ['1008079', '1008074'],
 1008175: ['1008197', '1881391', '1008192'],
 1008160: ['1008267', '1008262'],
 1008254: ['1008382', '1008376'],
 1008331: ['1008405', '1008410'],
 1008354: ['1008436', '100

In [ ]:
localisation[localisation.pk_person == 1013187]
# ['1013243', '1019481', '1013237']

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
117,1013237,1013187,"Alessandro Tasca (da Bergamo, marzer)",S. Marina (743097),Parrocchia,45.438499,12.339443,Abitazione,1597.0,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,1013243,1013187,"Alessandro Tasca (da Bergamo, marzer)",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,1586.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,1019481,1013187,"Alessandro Tasca (da Bergamo, marzer)",S. Silvestro (743202),Parrocchia,45.437638,12.333312,Abitazione,1586.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
localisation[localisation.pk_person != localisation.pk_person_x][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob', 'id_localisation_x',
       'pk_person_x','begin_x', 'at_some_time_x', 'end_x',
       'occours_before_x', 'occours_after_x', 'sin_ob_x']]

,id_localisation,pk_person,person,geographical_place,begin,at_some_time,end,occours_before,occours_after,sin_ob,id_localisation_x,pk_person_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),NaN,1597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1006467,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",S. Marziale (743039),NaN,1598.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),1591.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),1586.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1006544,1006430,Pietro di Giovanni Maria (da Venezia),S. Nicolò (743132),NaN,1597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14879,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),1580.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14880,999714,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",S. Eufemia (742937),NaN,1597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14881,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),1580.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14882,999734,999628,"Giacomo di Giulio Zambellini (da Bergamo, vend...",S. Geremia (742972),NaN,1597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
localisation[localisation.id_localisation=='1006461']

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
0,1006461,1006422,Giovanni di Michele (da Cessalto),S. Nicolò (743132),Parrocchia,45.432772,12.315848,Abitazione,NaN,1597.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
for i,e in loca.fillna(0).iterrows():
    if e.begin ==0:
        if e.at_some_time!=0:
            loca.begin[i] = e.at_some_time
        


C:\Users\Jacopo COSSU\AppData\Local\Temp/ipykernel_8084/1660512413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loca.begin[i] = e.at_some_time


In [ ]:
loca = loca[loca.begin.notnull() & loca.at_some_time.isnull()]; local = loca[['id_localisation', 'pk_person', 'person', 'geographical_place', 'begin', 'tag']]; display(local := processetti.merge(local, how='inner', on='pk_person'))

,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union,id_localisation,person,geographical_place,begin,tag
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),1006422.0,Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0,1006478,Giovanni di Michele (da Cessalto),Venezia IT (209678),1586.0,arrivo_venezia
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",999599.0,Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0,1006718,"Francesco di Vincenzo Moscato (da Fabriano, te...",Venezia IT (209678),1583.0,arrivo_venezia
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",1006794.0,Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0,1006877,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",Venezia IT (209678),1588.0,arrivo_venezia
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),1006856.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0,1007173,Orsola di Angelo Collotis (da Visinale),Venezia IT (209678),1585.0,arrivo_venezia
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),1007216.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0,1007350,Ortensia di Leonardo Verona (da Venzone),Venezia IT (209678),1588.0,arrivo_venezia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2761,EM 05,Processetto di Giacomo di Giovanni Capellari (...,1,998068,"Giacomo di Giovanni Capellari (da Casanova, te...",997852.0,Stato libero di “foresto”,1598-12-14,1598-12-14,1598.0,NaN,0,998062,"Giacomo di Giovanni Capellari (da Casanova, te...",Venezia IT (209678),1580.0,arrivo_venezia
2762,EM 05,Processetto di Antonio di Simone Marghirato (d...,1,998269,"Antonio di Simone Marghirato (da Travesio, fac...",998101.0,Stato libero di “foresto”,1598-12-16,1598-12-16,1598.0,NaN,0,998238,"Antonio di Simone Marghirato (da Travesio, fac...",Venezia IT (209678),1587.0,arrivo_venezia
2763,EM 05,"Processetto di Gaspare Forzanini (da Bassano, ...",1,998606,"Gaspare Forzanini (da Bassano, calegher)",998390.0,Stato libero di “foresto”,1598-12-18,1598-12-18,1598.0,NaN,0,998596,"Gaspare Forzanini (da Bassano, calegher)",Venezia IT (209678),1586.0,arrivo_venezia
2764,EM 05,Processetto di Andrea di Angelo Pasetto (da Ma...,1,999218,"Andrea di Angelo Pasetto (da Magno, erbariol)",998838.0,Stato libero di “foresto”,1598-12-21,1598-12-21,1598.0,NaN,0,999197,"Andrea di Angelo Pasetto (da Magno, erbariol)",Venezia IT (209678),1587.0,arrivo_venezia


--------------------------------------

### Chauvard (17/08/2022):

>   TODO: Sviluppa sulla base dei risultati un file per la bonifica

In [ ]:
local['periodo'] = local.anno - local.begin
local.groupby('motivation_type').agg({'periodo':'mean'})

,periodo
motivation_type,
Processetto atipico,3.000000
Stato libero di persona lungamente assente da Venezia,4.571429
Stato libero di persona lungamente assente da Venezia; Stato libero di “foresto”,12.904762
Stato libero di persona lungamente assente da Venezia; Stato libero di “foresto” ; Processetto atipico,9.500000
Stato libero di “foresto”,11.743295
Stato libero di “foresto” ; Processetto atipico,12.090909
Vedovanza,10.056180
Vedovanza; Processetto atipico,12.000000
Vedovanza; Stato libero di “foresto”,9.500000


In [ ]:
local['periodo'] = local.anno - local.begin
local.groupby('motivation_type').agg({'periodo':['mean', 'max', 'min']})

periodo             
                                                         mean    max   min
motivation_type                                                           
Processetto atipico                                  3.000000    5.0   0.0
Stato libero di persona lungamente assente da V...   4.571429   10.0   0.0
Stato libero di persona lungamente assente da V...  12.904762   29.0   0.0
Stato libero di persona lungamente assente da V...   9.500000   13.0   6.0
Stato libero di “foresto”                           11.743295  114.0  -5.0
Stato libero di “foresto” ; Processetto atipico     12.090909   32.0   1.0
Vedovanza                                           10.056180   30.0   0.0
Vedovanza; Processetto atipico                      12.000000   16.0  10.0
Vedovanza; Stato libero di “foresto”                 9.500000   30.0   1.0

In [ ]:
local[(local.periodo>50) | (local.periodo<0)]

,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union,id_localisation,person,geographical_place,begin,tag,periodo
175,EM 02,Processetto di Natalina di Nicola Colla (da Mo...,1,1074116,Natalina di Nicola Colla (da Montona),1074008.0,Stato libero di “foresto”,1594-02-06,1594-02-07,1594.0,NaN,0,1810547,Natalina di Nicola Colla (da Montona),Venezia IT (209678),1599.0,abitazione_venezia,-5.0
927,EM 25,Processetto di Gabriele di Bettino Gabrieli (d...,1,1486463,"Gabriele di Bettino Gabrieli (da Valle Sabbia,...",1486412.0,Stato libero di “foresto”,1627-08-08,1627-08-08,1627.0,NaN,0,1486487,"Gabriele di Bettino Gabrieli (da Valle Sabbia,...",Venezia IT (209678),1629.0,arrivo_venezia,-2.0
1202,EM 07,Processetto di Gaspare di Gaspare Ballestra (d...,1,1616092,"Gaspare di Gaspare Ballestra (da Leichlingen, ...",1614020.0,Stato libero di “foresto”,1693-02-04,1693-02-04,1693.0,NaN,0,1616087,"Gaspare di Gaspare Ballestra (da Leichlingen, ...",Venezia IT (209678),1599.0,arrivo_venezia,94.0
2262,EM 53,Processetto di Gerolamo di Giacomo Ruggeri (da...,1,2075549,"Gerolamo di Giacomo Ruggeri (da Bergamo, marzer)",2075472.0,Stato libero di “foresto”,1650-09-16,1650-09-16,1650.0,NaN,0,2075486,"Gerolamo di Giacomo Ruggeri (da Bergamo, marzer)",Venezia IT (209678),1536.0,arrivo_venezia,114.0
2447,EM 40,Processetto di Pietro di Angelo Carella (da Sa...,1,2192765,"Pietro di Angelo Carella (da Salgareda, fachin)",2192677.0,Stato libero di “foresto”,1630-02-24,1630-02-24,1630.0,NaN,0,2192692,"Pietro di Angelo Carella (da Salgareda, fachin)",Venezia IT (209678),1634.0,arrivo_venezia,-4.0


In [ ]:
df_ric[df_ric.pk_person==1486412]

,pk_processetto,pk_person,person,qualità,età,anno_nascita,genere,origini,professioni
1087,1486463,1486412,"Gabriele di Bettino Gabrieli (da Valle Sabbia,...",richiedente,21.0,1606.0,Maschile,Valle Sabbia,faber murarius


In [ ]:
a = local[[ 'pk_processetto', 'pk_person', 'motivation_type', 
       'anno',  'id_localisation', 
       'geographical_place', 'begin', 'tag', 'periodo']].merge(df_ric, on='pk_person', how='left')

In [ ]:
a.columns


Index(['pk_processetto_x', 'pk_person', 'motivation_type', 'anno',
       'id_localisation', 'geographical_place', 'begin', 'tag', 'periodo',
       'pk_processetto_y', 'person', 'qualità', 'età', 'anno_nascita',
       'genere', 'origini', 'professioni'],
      dtype='object')

In [ ]:
a['eta_arrivo_venezai'] = a.begin - a.anno_nascita; a.groupby(['motivation_type','eta_arrivo_venezai']).agg({'pk_processetto_x':'count','periodo':'mean'})

pk_processetto_x  \
motivation_type                                    eta_arrivo_venezai                     
Processetto atipico                                16.0                               1   
                                                   17.0                               2   
                                                   25.0                               2   
Stato libero di persona lungamente assente da V... 12.0                               1   
                                                   13.0                               1   
...                                                                                 ...   
Vedovanza                                          41.0                               1   
                                                   46.0                               1   
Vedovanza; Stato libero di “foresto”               25.0                               1   
                                                   29.0                               1   
                                                   35.0                               1   

                                                                       periodo  
motivation_type                                    eta_arrivo_venezai           
Processetto atipico                                16.0                    4.0  
                                                   17.0                    5.0  
                                                   25.0                    0.0  
Stato libero di persona lungamente assente da V... 12.0                   10.0  
                                                   13.0                    7.0  
...                                                                        ...  
Vedovanza                                          41.0                    9.0  
                                                   46.0                    2.0  
Vedovanza; Stato libero di “foresto”               25.0                    7.0  
                                                   29.0                   10.0  
                                                   35.0                   15.0  

[108 rows x 2 columns]

In [ ]:
a['controllo'] = a.età < a.periodo; a[a.controllo==True]

,pk_processetto_x,pk_person,motivation_type,anno,id_localisation,geographical_place,begin,tag,periodo,pk_processetto_y,person,qualità,età,anno_nascita,genere,origini,professioni,eta_arrivo_venezai,controllo
95,1054365,1054307.0,Stato libero di “foresto”,1597.0,1054359,Venezia IT (209678),1552.0,arrivo_venezia,45.0,1054365.0,"Giovanni di Nicola Valentini (da Ampezzo, frut...",richiedente,23.0,1574.0,Maschile,Ampezzo di Cadore,frutariol,-22.0,True
201,1079648,1078862.0,Stato libero di “foresto”,1594.0,1078890,Venezia IT (209678),1582.0,arrivo_venezia,12.0,1079648.0,"Orazio di Cesare Bertelli (da Roma, sartor)",richiedente,8.0,1586.0,Maschile,Roma IT,sartor,-4.0,True
1005,1511912,1511837.0,Stato libero di “foresto”,1602.0,1511907,Venezia IT (209678),1586.0,arrivo_venezia,16.0,1511912.0,"Andrea di Pietro Armi (da Bassano, sartor)",richiedente,8.0,1594.0,Maschile,Bassano,sartor,-8.0,True
1085,1550006,1549924.0,Stato libero di “foresto”,1602.0,1550000,Venezia IT (209678),1587.0,arrivo_venezia,15.0,1550006.0,Felicita di Pietro Lorenzi (da San Nicolò di C...,richiedente,8.0,1594.0,Maschile,San Nicolò di Comelico,masser,-7.0,True
1295,1647730,1647317.0,Stato libero di “foresto”,1603.0,1647725,Venezia IT (209678),1583.0,arrivo_venezia,20.0,1647730.0,Costantino di Pietro Buttono (da Cipro),richiedente,18.0,1585.0,Maschile,Cipro,NaN,-2.0,True
1886,1876132,1876049.0,Stato libero di “foresto”,1595.0,1876127,Venezia IT (209678),1556.0,arrivo_venezia,39.0,1876132.0,"Leonardo di Matteo Pontiloni (da Vito d'Asio, ...",richiedente,31.0,1564.0,Maschile,Vito d'Asio,remer,-8.0,True
2268,2075549,2075472.0,Stato libero di “foresto”,1650.0,2075486,Venezia IT (209678),1536.0,arrivo_venezia,114.0,2075549.0,"Gerolamo di Giacomo Ruggeri (da Bergamo, marzer)",richiedente,40.0,1610.0,Maschile,Bergamo,marzer,-74.0,True


In [ ]:
local[local.begin.notnull()].groupby('motivation_type').agg({'pk_processetto':'count'})

,pk_processetto
motivation_type,
Processetto atipico,3
Stato libero di persona lungamente assente da Venezia,7
Stato libero di persona lungamente assente da Venezia; Stato libero di “foresto”,21
Stato libero di persona lungamente assente da Venezia; Stato libero di “foresto” ; Processetto atipico,2
Stato libero di “foresto”,2616
Stato libero di “foresto” ; Processetto atipico,11
Vedovanza,89
Vedovanza; Processetto atipico,3
Vedovanza; Stato libero di “foresto”,12


In [ ]:
loca

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after,tag,note
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1591.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
5,1006553,1006430,Pietro di Giovanni Maria (da Venezia),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1547.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
7,1006588,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
10,1006634,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1589.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14871,999021,998887,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14873,999197,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN,arrivo_venezia,NaN


-------------------------------

In [ ]:
processetti

,volume,section,number_archival_units,pk_processetto,enquiry,pk_person,motivation_type,start_date,end_date,anno,id_union,number_union
0,EM 04,Processetto di Giovanni di Michele (da Cessalto),1,1006519,Giovanni di Michele (da Cessalto),1006422.0,Stato libero di “foresto”,1597-05-23,1597-05-23,1597.0,NaN,0
1,EM 05,Processetto di Francesco di Vincenzo Moscato (...,1,1006729,"Francesco di Vincenzo Moscato (da Fabriano, te...",999599.0,Stato libero di “foresto”,1598-12-27,1598-12-27,1598.0,NaN,0
2,EM 04,Processetto di Pietro di Gennaro Costinovi (da...,1,1006883,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",1006794.0,Stato libero di “foresto”,1597-05-26,1597-05-26,1597.0,NaN,0
3,EM 05,Processetto di Orsola di Angelo Collotis (da V...,1,1007147,Orsola di Angelo Collotis (da Visinale),1006856.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
4,EM 05,Processetto di Ortensia di Leonardo Verona (da...,1,1007355,Ortensia di Leonardo Verona (da Venzone),1007216.0,Stato libero di “foresto”,1598-12-29,1598-12-29,1598.0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,EM 05,"Processetto di Gaspare Forzanini (da Bassano, ...",1,998606,"Gaspare Forzanini (da Bassano, calegher)",998390.0,Stato libero di “foresto”,1598-12-18,1598-12-18,1598.0,NaN,0
3793,EM 04,Processetto di Cristina vedova di Nicola De Ga...,1,998615,Cristina vedova di Nicola De Gaspari (da Venezia),998555.0,Vedovanza,1597-05-20,1597-05-20,1597.0,998630,1
3794,EM 04,Processetto di Giovanni Maria di Antonio (da L...,1,998913,"Giovanni Maria di Antonio (da Lovere, tesser d...",998798.0,Vedovanza,1597-05-21,1597-05-21,1597.0,998922,1
3795,EM 05,Processetto di Andrea di Angelo Pasetto (da Ma...,1,999218,"Andrea di Angelo Pasetto (da Magno, erbariol)",998838.0,Stato libero di “foresto”,1598-12-21,1598-12-21,1598.0,NaN,0


In [ ]:
localisation.columns

Index(['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after', 'sin_ob', 'id_localisation_x',
       'pk_person_x', 'person_x', 'geographical_place_x', 'GP_type_x', 'lat_x',
       'lon_x', 'localisation_type_x', 'begin_x', 'at_some_time_x', 'end_x',
       'occours_before_x', 'occours_after_x', 'sin_ob_x'],
      dtype='object')

In [ ]:
localisation[localisation.geographical_place.notnull() & localisation.geographical_place.str.contains('209678') & localisation.at_some_time.notnull() & (localisation.occours_before.notnull() | localisation.occours_after.notnull())][['id_localisation', 'pk_person', 'person', 'geographical_place',
       'GP_type', 'lat', 'lon', 'localisation_type', 'begin', 'at_some_time',
       'end', 'occours_before', 'occours_after']]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
4875,1479884,1479800,Bartolomeo di Antonio Stini (da Colmaggiore),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,1604.0,NaN,NaN,1479911


In [ ]:
loc[(loc.pk_person==1890230)]

AttributeError: 'str' object has no attribute 'pk_person'

In [ ]:
localisation[(localisation.pk_person==1890230)]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,...,GP_type_x,lat_x,lon_x,localisation_type_x,begin_x,at_some_time_x,end_x,occours_before_x,occours_after_x,sin_ob_x
10112,1890243,1890230,"Domenico di Leonardo Franceschi (da Pordenone,...",S. Maria Nova (741882),Parrocchia,45.439623,12.339009,Abitazione,NaN,1595.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10113,1890248,1890230,"Domenico di Leonardo Franceschi (da Pordenone,...",Venezia IT (209678),Città,45.437130,12.332650,Abitazione,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Arrivo a Venezia:

gp ==> Venezia IT (209678)

type ==> Città

__loc_type ==> Abitazione (?)__

ast ==> null() __[or notnull() if b.null()](?)__

e ==> null()

ob ==> rilevante 

oa ==> non rilevante

In [ ]:
(venezia := loc[loc.geographical_place.notnull() & loc.geographical_place.str.contains('Venezia') & loc.GP_type.str.contains('(?i)città')])

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1591.0,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,NaN,NaN
5,1006553,1006430,Pietro di Giovanni Maria (da Venezia),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1547.0,NaN,NaN,NaN,NaN
7,1006588,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN
10,1006634,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1589.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14871,999021,998887,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN
14873,999197,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN


In [ ]:
venezia[venezia.begin.isnull() & venezia.at_some_time.isnull() &venezia.end.isnull() & (venezia.occours_after.notnull()|venezia.occours_before.notnull())]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
2038,1123170,1123151,"Domenico di Aloisio Aprilis (da Cusano, barcar...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1123094],NaN
2134,1127958,1127939,"Giovanni di Francesco (da Pozzo, petener)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1127918],NaN
2175,1129329,1129308,"Simone di Giovanni Vinea (da Chiavenna, strama...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1129294],NaN
3363,1249229,1249204,"Francesco di Bettino Colzadi (bergamasco, port...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1249192],NaN
3813,1277652,1277608,Francesco di Filippo Coltrer (milanese),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1277644],NaN
4078,1298688,1128418,"Paolo di Domenico Zucchetti (bresciano, murer)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1128368],NaN
4314,1320503,1320478,"Giacomo di Bettino Silvestri (vicentino, camer...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1320545],NaN
4527,1340918,1340880,"Francesco di Filippo De Nottis (da Arba, teraz...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1340857],NaN
5586,1544809,1544770,"Antonio di Bonetto (bergamasco, frutariol)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1544755],NaN
7565,1700726,1700695,Giovanni Battista di Cesare Marcellino (dal La...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,NaN,NaN,NaN,[1573019],NaN


>   Il dataset rappresenta arrivi a Venezia periodizzati unicamente sulla base di altri event

per questi è necessario definire una strategia per la datazione

Archi aperti: Begin but not end >> Rappresenta un arrivo a venezia, ammesso che non abbia compilato l'occours_before (con identico pk_person e il riferimento a un'entità con GP_type >= Città)

ast >> Se coincidente con la data del processetto `arrivo a Venezia` altrimenti `soggiorno a Venezia`

Archi chiusi: Begin and end >> Rappresenta un avvenimento circoscritto nel tempo (non dovrebbe essere assunto come arrivo a Venezia) [si può indicare come `soggiorno in città`]

Senza archi:
    -   Privo di periodizzazione >> Valuta casi >> Verifica la possibilità di indicare come ast l'anno del processetto 
    -   oa and ob >> `soggiorno in città`
    -   oa but not ob >> si può assumere che rappresenti l'ulima indicazione -- attribuire la data del processetto come ast 
    -   ob but not oa >> `soggiorno in città`



-   costruire un df integrativo per la gestione dei occours before

In [ ]:
df_ric

,pk_processetto,pk_person,person,qualità,età,anno_nascita,genere,origini,professioni
0,1006519,1006422,Giovanni di Michele (da Cessalto),richiedente,21.0,1576.0,Maschile,Cessalto,NaN
1,1006883,1006794,"Pietro di Gennaro Costinovi (da Bergamo, tesse...",richiedente,21.0,1576.0,Maschile,Bergamo,cogo; tesser da panni
2,1007147,1006856,Orsola di Angelo Collotis (da Visinale),richiedente,26.0,1572.0,Femminile,Visinale,masser
3,1007355,1007216,Ortensia di Leonardo Verona (da Venzone),richiedente,25.0,1573.0,Femminile,Venzone,masser
4,1007563,1007388,"Vincenzo di Battista Betocchi (da Gandino, cas...",richiedente,26.0,1572.0,Maschile,Gandino,casariol
...,...,...,...,...,...,...,...,...,...
3424,998615,998555,Cristina vedova di Nicola De Gaspari (da Venezia),richiedente,NaN,NaN,Femminile,Venezia IT,NaN
3425,998913,998798,"Giovanni Maria di Antonio (da Lovere, tesser d...",richiedente,NaN,NaN,Maschile,Lovere,tesser da panni
3426,999218,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",richiedente,25.0,1573.0,Maschile,Magno,erbariol; naranzer
3427,999679,999563,Giovanni di Antonio Maiti (da Bergamo),richiedente,33.0,1564.0,Maschile,Bergamo,NaN


In [ ]:
## Archi aperti

display(archi_aperti := venezia[venezia.begin.notnull() & venezia.end.isnull()])

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
2,1006473,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1591.0,NaN,NaN,NaN,NaN
3,1006478,1006422,Giovanni di Michele (da Cessalto),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,NaN,NaN
5,1006553,1006430,Pietro di Giovanni Maria (da Venezia),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1547.0,NaN,NaN,NaN,NaN
7,1006588,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN
10,1006634,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1589.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14871,999021,998887,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN
14873,999197,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,NaN,NaN
14875,999673,999563,Giovanni di Antonio Maiti (da Bergamo),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN
14877,999719,999580,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1580.0,NaN,NaN,NaN,NaN


In [ ]:
archi_aperti[archi_aperti.occours_before.notnull()]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
4247,1305107,1305082,Battista di Giovanni Maria Ambrogi (dalla Val ...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,[1305069],NaN
4800,1473728,1473708,Bernardino di Giacomo Di Bortolo (da San Giorg...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1586.0,NaN,NaN,[1473761],NaN
5305,1509981,1509601,"Giovanni di Francesco Brunelli (da Bologna, st...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1607.0,NaN,NaN,[1510141],NaN
10107,1890217,1890192,"Giorgio di Romano Candioti (da Pordenone, tess...",Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1584.0,NaN,NaN,[1890248],NaN
14417,918322,764913,Ventura di Taddeo Sarzeli (da Romano di Lombar...,Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1578.0,NaN,NaN,[942564],NaN
14539,926345,779471,Lancillotto di Pietro Martino Rusca (da Lugano),Venezia IT (209678),Città,45.43713,12.33265,Abitazione,1587.0,NaN,NaN,[779555],NaN
14569,927366,777519,Domenica (massera),Venezia IT (209678),Città,45.43713,12.33265,Abitazione e luogo di lavoro,1592.0,NaN,NaN,[777601],NaN


In [ ]:
loc[loc.id_localisation==779555]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after


In [ ]:
loc[loc.at_some_time.isnull() & loc.end.isnull() & loc.occours_after.isnull() & loc.occours_before.isnull() & (loc.GP_type!='Città')]

,id_localisation,pk_person,person,geographical_place,GP_type,lat,lon,localisation_type,begin,at_some_time,end,occours_before,occours_after
258,1038233,1036210,Giovanna (da Zara),S. Maria Formosa (743074),Parrocchia,45.436900,12.341100,Abitazione,1578.0,NaN,NaN,NaN,NaN
268,1038695,1038567,"Domenico di Giovanni Maria (da Dosson, tentor)",S. Maria Formosa (743074),Parrocchia,45.436900,12.341100,Luogo di lavoro,NaN,NaN,NaN,NaN,NaN
297,1040012,1039749,"Gregorio di Domenico (da Monfalcone, sartor)",Angelo Raffael (743174),Parrocchia,45.432506,12.318992,Luogo di lavoro,1590.0,NaN,NaN,NaN,NaN
392,1044971,1044706,Antonia di Marco Tognetti (da Castelfranco Ven...,Cannaregio (945713),Sestiere,45.446000,12.327000,Abitazione,1587.0,NaN,NaN,NaN,NaN
415,1046376,1046322,Defendi di Pietro Bruni (da Valle San Martino),S. Pantalon (743139),Parrocchia,45.435720,12.324740,Abitazione,1584.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14634,929686,754178,Pasqua di Gregorio Celodo (da Santa Lucia),S. Croce (742475),Parrocchia,45.438927,12.320719,Luogo di lavoro,1577.0,NaN,NaN,NaN,NaN
14706,987168,987013,Giovanni di Prospero Biagi (da Mariano del Fri...,S. Moisè (743125),Parrocchia,45.433056,12.336111,Luogo di lavoro,1582.0,NaN,NaN,NaN,NaN
14814,993891,993836,Caterina di Tommaso Brunelli,S. Angelo (742839),Parrocchia,45.433953,12.331963,Abitazione,1584.0,NaN,NaN,NaN,NaN
14835,996202,994175,Giacomo di Simone Coriscelli (da Forni di Sopr...,S. Marco (743032),Parrocchia,45.434550,12.339500,Luogo di lavoro,1587.0,NaN,NaN,NaN,NaN


In [ ]:
df_loc.describe(exclude=np.number)

,person,geographical_place,GP_type,localisation_type,occours_before,occours_after
count,14880,14868,14861,14828,172,169
unique,9508,227,7,5,170,169
top,Lucrezia Sanudo (capodistriana),Venezia IT (209678),Parrocchia,Abitazione,1084859,1012100
freq,6,5229,9151,13818,2,1


### Occupation (event)

In [ ]:
file = 'geovistory_query_exports/nb_occupation_event_20220512.json'

with open(file, encoding='utf-8') as json_file:
    data_occup = json.load(json_file)

dmoe = DotMap(data_occup)
dmoe_r = dmoe.rows

In [ ]:
print(data_occup['columns'], '\n')
dmoe_r[0].col_3

for r in dmoe_r:
    if len(r.col_5['values'])>0:
        print(r.col_5)
        break


{'col_0': 'Person', 'col_1': 'Occupation (event)', 'col_2': 'Occupation', 'col_3': 'Begin', 'col_4': 'End', 'col_5': 'At some time', 'col_6': 'Occurs before', 'col_7': 'Occurs after'} 

DotMap(values=[DotMap(value=DotMap(timePrimitive=DotMap(to=DotMap(calJulian='1599-01-22', julianDay=2305114, calGregorian='1599-02-01', julianSecond=199161849600, calGregorianIso8601='1599-02-01T00:00:00Z'), from=DotMap(calJulian='1598-12-22', julianDay=2305083, calGregorian='1599-01-01', julianSecond=199159171200, calGregorianIso8601='1599-01-01T00:00:00Z'), label='1599-01-01 (1 month)', fkClass=335, calendar='gregorian', duration='1 month', pkEntity=1007929, julianDay=2305083)), pkStatement=1008309, fkObjectInfo=1007929, fkSubjectInfo=1008293)])


In [ ]:
# Occupation event list = oel

oel = [['pk_person', 'person', 'id_occup_event', 'occupation', 'begin', 'at_some_time', 'end', 'occours_before', 'count_OB', 'occours_after', 'count_OA']]

for r in dmoe_r:
    element = []

    # pk_person, person
    if len(r.col_0.entities)>0:
        element += [str(r.col_0.entities[0].pk_entity) , clean_up_str(r.col_0.entities[0].entity_label)]
    else:
        element += [np.nan, np.nan]
    
    # id occupation event, occupation
    element += [r.col_1.entity.pk_entity, r.col_2.entities[0].entity_label if len(r.col_2.entities)>0 else np.nan]

    # Begin, At some time within, End
    datatio(r.col_3, r.col_5, r.col_4, element)

    '''
    # Begin
    # riscrivi
    if len(r.col_3['values'])==1:
        element += [int(r.col_3['values'][0].value.timePrimitive.label[:4])]
    elif len(r.col_3['values'])>1:
        element += [round(np.mean([int(r.col_3['values'][0].value.timePrimitive.label[:4]), int(r.col_3['values'][1].value.timePrimitive.label[:4])]))]
    else:
        element += [np.nan]
    
    
    # At some time within
    element += [int(r.col_5['values'][0].value.timePrimitive.label[:4]) if len(r.col_5['values'])!=0 else np.nan]

    # End
    ## riscrivi
    if len(r.col_4['values'])==1:
        element += [int(r.col_4['values'][0].value.timePrimitive.label[:4])]
    elif len(r.col_4['values'])>1:
        element += [round(np.mean([int(r.col_4['values'][0].value.timePrimitive.label[:4]), int(r.col_4['values'][1].value.timePrimitive.label[:4])]))]
    else:
        element += [np.nan]
    '''
    
    # Occours before, conteggio (OB),  Occours after, conteggio(OA)
    element += ['-'.join(str(v.pk_entity) for v in r.col_6.entities) if len(r.col_6.entities)>0 else np.nan, len(r.col_6.entities), '-'.join(str(v.pk_entity) for v in r.col_7.entities) if len(r.col_7.entities)>0 else np.nan, len(r.col_7.entities)]
    
    oel.append(element)
    
df_oce = pd.DataFrame(oel[1:], columns=oel[0])
df_oce.head()



,pk_person,person,id_occup_event,occupation,begin,at_some_time,end,occours_before,count_OB,occours_after,count_OA
0,999827,"Angelo di Domenico Venturino (da Fabriano, mer...",1006497,mercante,NaN,NaN,NaN,NaN,0,NaN,0
1,1006438,"Antonio di Ambrogio Didano (da Bergamo, vender...",1006594,vender vin,NaN,NaN,NaN,NaN,0,NaN,0
2,1006449,"Frizerio di Antonio Zuliani (da Venezia, pesca...",1006620,NaN,NaN,NaN,NaN,NaN,0,NaN,0
3,1002990,"Giuseppe di Francesco Domiziano (da Fabriano, ...",1006641,cimador da panni,NaN,NaN,NaN,NaN,0,NaN,0
4,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",1006738,tentor,NaN,NaN,NaN,NaN,0,NaN,0


In [ ]:
df_oce.describe(exclude=np.number)

,pk_person,person,occupation,occours_before,occours_after
count,8174,8174,8116,97,97
unique,7878,7875,506,95,95
top,1148703,Battista Melada (portador da vin),mariner,1046024,1148837
freq,4,4,632,2,3


### Union

In [ ]:
file = 'geovistory_query_exports/nb_union_20220517.json'

with open(file, encoding='utf-8') as json_file:
    data_union = json.load(json_file)

dmu = DotMap(data_union)

dmu_r = dmu.rows

In [ ]:
print(data_union['columns'])
dmu_r[7].col_2

{'col_0': 'Pk_Processetto_di', 'col_1': 'Entity Preview', 'col_2': 'Partner', 'col_3': 'Union_type', 'col_4': 'Begin', 'col_5': 'Some_Time', 'col_6': 'End', 'col_7': 'Stem'}


DotMap(entities=[DotMap(key='591_1038812', project=591, fk_class=21, pk_entity=1038812, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Antonio di Giacomo Dalla Valle (da Cerete Basso)', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00'), DotMap(key='591_1039703', project=591, fk_class=21, pk_entity=1039703, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Margherita di Baldo Balzar (da Cerete Basso)', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')], _ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [ ]:
# funzione particolare 

def individuazione_partner(r_col, element):
    
    if len(r_col.entities)>1:
                for p in r_col.entities:
                    if p.pk_entity != element[2]:
                       element += [p.pk_entity, clean_up_str(p.entity_label)]
    else:
        element += [np.nan, 'missing']
    return(element)



# Union list = ul

ul = [['pk_processetto', 'id_union', 'pk_person',  'person', 'qualità', 'pk_secondPerson', 'secondPerson', 'begin', 'at_some_time', 'end']]

for r in dmu_r:
    
    if len(r.col_0.entities)>0:
        for e in r.col_0.entities:
            element = [] 
            element += [e.pk_entity, r.col_1.entity.pk_entity]
            for p in r.col_2.entities:
                if p.entity_label == e.entity_label:
                    element += [p.pk_entity, clean_up_str(p.entity_label), 'richiedente']
            individuazione_partner(r.col_2, element)
            datatio(r.col_4, r.col_5, r.col_6, element)
            ul.append(element)          
    else:
        element = [np.nan, r.col_1.entity.pk_entity, r.col_2.entities[0].pk_entity if len(r.col_2.entities)>0 else np.nan, clean_up_str(r.col_2.entities[0].entity_label) if len(r.col_2.entities)>0 else 'missing', 'altro']
        individuazione_partner(r.col_2, element)
        datatio(r.col_4, r.col_5, r.col_6, element)
        ul.append(element) 
    
df_union = pd.DataFrame(ul[1:],columns=ul[0])
df_union.head()

,pk_processetto,id_union,pk_person,person,qualità,pk_secondPerson,secondPerson,begin,at_some_time,end
0,1007675.0,1007670,1007492.0,Santa vedova di Battista di Nicola,richiedente,1007544.0,Battista di Nicola,NaN,NaN,1595.0
1,1007777.0,1007767,1007715.0,Filippo Amis (da Bergamo),richiedente,1007750.0,Angela (da Bergamo),NaN,NaN,1596.0
2,1026527.0,1026547,1012979.0,Caterina Albrizzi vedova di Francesco Bettinel...,richiedente,1013001.0,Francesco Bettinelli (curamer),NaN,NaN,1591.0
3,1032502.0,1032511,1032116.0,Bradamante di Francesco Canano (da Pordenone) ...,richiedente,1032159.0,Marco Antonio di Francesco Galeotto,1575.0,NaN,1590.0
4,1036286.0,1038311,1036219.0,Bernardo di Virgilio Cauzio (da Velle di Ledro...,richiedente,NaN,missing,NaN,NaN,NaN


In [ ]:
df_union.describe(exclude=np.number)

,person,qualità,secondPerson
count,558,558,558
unique,545,2,529
top,missing,richiedente,missing
freq,5,538,27


In [ ]:
df_union[df_union.person=='missing']

,pk_processetto,id_union,pk_person,person,qualità,pk_secondPerson,secondPerson,begin,at_some_time,end
56,NaN,1108833,NaN,missing,altro,NaN,missing,NaN,NaN,1597.0
266,NaN,1789911,NaN,missing,altro,NaN,missing,NaN,NaN,NaN
529,NaN,837480,NaN,missing,altro,NaN,missing,NaN,NaN,NaN
530,NaN,873810,NaN,missing,altro,NaN,missing,NaN,NaN,NaN
542,NaN,927723,NaN,missing,altro,NaN,missing,1595.0,NaN,NaN


### Social Relationship

In [ ]:
file = 'geovistory_query_exports/nb_social_relationship_20220512.json'

with open(file, encoding='utf-8') as json_file:
    data_srel = json.load(json_file)

dmsr = DotMap(data_srel)
dmsr_r = dmsr.rows

In [ ]:
print(data_srel['columns'], '\n')
dmsr_r[0].col_2

{'col_0': 'Pk_Processetto_di', 'col_1': 'Richiedente', 'col_2': 'Defunto', 'col_3': 'Union', 'col_4': 'Social relationship', 'col_5': 'Social relationship type', 'col_6': 'Partners (symmetrical)', 'col_7': 'Relationship source', 'col_8': 'Relationship target', 'col_9': 'Begin', 'col_10': 'At some time', 'col_11': 'End'} 



DotMap(entities=[], _ipython_display_=DotMap(), _repr_mimebundle_=DotMap())

In [ ]:
for r in dmsr_r:
    if len(r.col_8['entities'])>1:
        print(r.col_8)
        break

DotMap(entities=[DotMap(key='591_1079098', project=591, fk_class=21, pk_entity=1079098, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Gregorio di Martino Maretti (profumier)', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00'), DotMap(key='591_1079113', project=591, fk_class=21, pk_entity=1079113, fk_project=591, class_label='Person', entity_type='peIt', entity_label='Elisabetta di Leonardo Ventura (profumier)   ', tmsp_last_modification='2022-05-05T06:40:28.316247+00:00')])


**I**

| pk_proc | richiedente | defunto | ruolo nella relazione | altro partner | source | target | 
|--|--|--|--|--|--|--|
| 1:n | `null` | `null` | `null` | [*] | 0 | 0 
| 1:n | `null` | `null` | `null` | 0 | 1 | 1 | | 
| 1:n | 0 ; 1 | 1 ; 0 | 'partner' | 1 | 0 | 0 | | 
| 1:n | 0 ; 1 | 1 ; 0 | 'source' | 0 | 0 | 1 | | 
| 1:n | 0 ; 1 | 1 ; 0 | 'target' | 0 | 1 | 0 | |

<br>

**II**

| pk_proc | pk_person | qualità | ruolo nella relazione | partner (1) | partner(2) | source | target | descrizione |
|--|--|--|--|--|--|--|--|--|


<br>

**III**

|pk_proc | pk_person | qualità | ruolo nella relazione | partner/source | partner/source | tipo di relazione |
|--|--|--|--|--|--|--|
| 1:n | 0 ; 1 | richiedente/defunto/altro | partner/source/target | 1 | 1 | simmetrica/direzionata | 

<br>

**IV**

|pk_proc | qualità | person | ruolo nella relazione | person | ruolo nella relazione | tipo di relazione |
|--|--|--|--|--|--|--|
| 1 |   dfunto/testimone/altro | [person] |   partner/source/target   | [person] | partner/source/target    | simmetrica/direzionata    |

#### Riscrivi:

In [ ]:
# Da riscrivere. Il flusso può essere razionalizzato e snellito 

# funzione particolare

def integratio_sr(element):    
    if len(r.col_6.entities)==2:
        for el in r.col_6.entities:
                if (el.pk_entity) != element[1]:
                    element += ['partner', el.pk_entity, clean_up_str(el.entity_label)]
    elif len(r.col_6.entities)==1:
        if (r.col_6.entities[0].pk_entity) != element[1]:
            element += ['error', r.col_6.entities[0].pk_entity, clean_up_str(r.col_6.entities[0].entity_label)]
        else:
            if len(r.col_7.entities) >0:
                element += ['error', r.col_7.entities[0].pk_entity, clean_up_str( r.col_7.entities[0].entity_label)]
            else:
                if len(r.col_8.entities) >0:
                    element += ['error', r.col_8.entities[0].pk_entity, clean_up_str( r.col_8.entities[0].entity_label)]
                else:
                    element += ['error', np.nan, np.nan]
    elif len(r.col_6.entities) >2:
        i=0
        while i in range(len(r.col_6.entities)):
            for el in r.col_6.entities:
                if (el.pk_entity) != element[1]:
                    element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                    break
                else:
                    i+=1
            break
        if i == len(r.col_6.entities):
            element += ['error', r.col_6.entities[0].pk_entity, clean_up_str(r.col_6.entities[0].entity_label)]
    else:
        if len(r.col_7.entities)>0:
            if len(r.col_7.entities) ==1:
                if (r.col_7.entities[0].pk_entity) != element[1]:
                    element += ['target', r.col_7.entities[0].pk_entity, clean_up_str(r.col_7.entities[0].entity_label)]
                else:
                    if len(r.col_8.entities)>0:     # ridondante
                        if len(r.col_8.entities)==1:
                            if (r.col_8.entities[0].pk_entity) != element[1]:
                                element += ['source', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
                            else: 
                                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
                        elif len(r.col_8.entities)>1:
                            i=0
                            while i in range(len(r.col_8.entities)):
                                for el in r.col_8.entities:
                                    if (el.pk_entity) != element[1]:
                                        element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                                        break
                                    else:
                                        i+=1
                                break
                            if i == len(r.col_8.entities):
                                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
            elif len(r.col_7.entities) > 1:
                i=0
                while i in range(len(r.col_7.entities)):
                    for el in r.col_7.entities:
                        if (el.pk_entity) != element[1]:
                            element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                            break
                        else:
                            i+=1
                    break
                if i == len(r.col_7.entities):
                    element += ['error', r.col_7.entities[0].pk_entity, clean_up_str(r.col_7.entities[0].entity_label)]
        else:
            if len(r.col_8.entities)>1:
                i=0
                while i in range(len(r.col_8.entities)):
                    for el in r.col_8.entities:
                        if (el.pk_entity) != element[1]:
                            element += ['error', el.pk_entity, clean_up_str(el.entity_label)]
                            
                            break
                        else:
                            i+=1
                    break
                if i == len(r.col_7.entities):
                    element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]
            elif len(r.col_8.entities)==0:
                element += ['error', np.nan, np.nan]
            else:
                element += ['error', r.col_8.entities[0].pk_entity, clean_up_str(r.col_8.entities[0].entity_label)]

    element += [r.col_4.entity.pk_entity, r.col_5.entities[0].entity_label if len(r.col_5.entities)>0 else np.nan]

     

# Social Relationship list = srl

srl =[['pk_processetto', 'pk_person', 'person', 'qualità', 'ruolo', 'pk_secondPerson', 'secondPerson', 'id_relazione', 'relation_type', 'begin', 'at_some_time', 'end']]


# pk_processetto/pk_processetto(union), pk_person, person_name, qualità
## Essendo la query definita a partire dalle Social Relationship, possono essere riportati più enquiry di validita (es. quando la relazione intercorre tra due richiedenti)
for r in dmsr_r:
    i = 0
    if len(r.col_0.entities) >= 1:
        for el in r.col_0.entities:
            element = []
            element += [el.pk_entity, r.col_1.entities[i].pk_entity if len(r.col_1.entities)>i else r.col_1.entities[(i-1)].pk_entity, clean_up_str(r.col_1.entities[(i)].entity_label) if len(
                r.col_1.entities)>i else clean_up_str(r.col_1.entities[(i-1)].entity_label), 'richiedente']
            i+=1
            integratio_sr(element)
            datatio(r.col_9, r.col_10, r.col_11, element)
            srl.append(element)
# Una relazione nei casi di 'vedovanza' può essere riferita a un'enquiry per mezzo dell'union
    else:   # (if len(r.col_0.entities) == 0:)
        if len(r.col_3.entities) >= 1:
            for el in r.col_3.entities:
                element = []
                element += [el.pk_entity]
                if len(r.col_2.entities) ==1:
                    element += [r.col_2.entities[0].pk_entity, clean_up_str(r.col_2.entities[0].entity_label), 'defunto']
# A causa di un difetto della query di partenza possono figurare più defunti; in tal caso è necessario individuare il corretto soggetto della relazione
                elif len(r.col_2.entities) >1:
                    lista_soggetti = []
                    if len(r.col_6.entities)>0:
                        for el in r.col_6.entities:
                            lista_soggetti.append(el.pk_entity)
                        for e in r.col_2.entities:
                            if e.pk_entity in lista_soggetti:
                                element += [e.pk_entity, clean_up_str(e.entity_label), 'defunto']
                    else:
                        for el in r.col_7.entities:
                            lista_soggetti.append(el.pk_entity) 
                        for el in r.col_8.entities:
                            lista_soggetti.append(el.pk_entity)
                        if e.pk_entity in lista_soggetti:
                                element += [e.pk_entity, clean_up_str(e.entity_label), 'defunto']
                        else:
                            element += ['error', np.nan, np.nan]
                else:
                    if len(r.col_6.entities)>0:
                        element += [r.col_6.entities[0].pk_entity, r.col_6.entities[0].entity_label, 'error']
                    else:
                        element += [r.col_7.entities[0].pk_entity if len(r.col_7.entities)>0 else r.col_8.entities[0].pk_entity if len(
                            r.col_8.entities)>0 else np.nan, r.col_7.entities[0].entity_label if len(r.col_7.entities)>0 else r.col_8.entities[0].entity_label if len(
                                r.col_8.entities)>0 else np.nan, 'error']
                integratio_sr(element)
                datatio(r.col_9, r.col_10, r.col_11, element)
                srl.append(element)
        else:
            element = [np.nan]
            if len(r.col_6.entities)>0:
                element += [r.col_6.entities[0].pk_entity, r.col_6.entities[0].entity_label, 'altro']
            else:
                if len(r.col_7.entities)>0:
                    element += [r.col_7.entities[0].pk_entity, r.col_7.entities[0].entity_label, 'altro']
                else:
                    if len(r.col_8.entities)>0:
                        element += [r.col_8.entities[0].pk_entity, r.col_8.entities[0].entity_label, 'altro']
                    else:
                        # assenza di individui coinvolti nella relazione --> errore di compilazione
                        element += ['error', np.nan, np.nan]
            integratio_sr(element)
            datatio(r.col_9, r.col_10, r.col_11, element)
            srl.append(element)
    


df_social = pd.DataFrame(srl[1:], columns=srl[0])
df_social


,pk_processetto,pk_person,person,qualità,ruolo,pk_secondPerson,secondPerson,id_relazione,relation_type,begin,at_some_time,end
0,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,partner,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006504,Origine comune,NaN,NaN,NaN
1,776800.0,776682,Lorenzo di Sebastiano Collo (bellunese),richiedente,source,776696.0,"Gottardo di Pietro Blasso (da Selva di Cadore,...",1006510,Vicinanza a/frequentazione di parenti e/o affini,1567.0,NaN,NaN
2,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,partner,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006514,Frequentazione occasionale,NaN,NaN,NaN
3,1006729.0,999599,"Francesco di Vincenzo Moscato (da Fabriano, te...",richiedente,source,999827.0,"Angelo di Domenico Venturino (da Fabriano, mer...",1006528,Vicinanza a/frequentazione di parenti e/o affini,NaN,NaN,NaN
4,776302.0,776078,Bartolomeo Evangelista Giustinelli (da Bornato),richiedente,partner,776131.0,"Tommaso di Battista Cineis (da Bornato, murer)",1006549,"Conoscenza, frequentazione e pratica",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9583,998913.0,998830,Gada di Comini (bergamasca),defunto,partner,998871.0,Maria Elisabetta moglie di Andrea Panigado (da...,999005,Vicinato abitativo nel paese d'origine,NaN,NaN,NaN
9584,999218.0,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",richiedente,partner,998852.0,"Innocenzo di Paolo Giorgi (da Dosso, erbariol)",999009,Parentela e/o affinità,1578.0,NaN,NaN
9585,999218.0,998838,"Andrea di Angelo Pasetto (da Magno, erbariol)",richiedente,partner,998887.0,"Giacomo di Bernardo Vivenzi (da Lavino, lavora...",999063,Parentela e/o affinità,NaN,NaN,NaN
9586,999679.0,999563,Giovanni di Antonio Maiti (da Bergamo),richiedente,partner,999580.0,"Vincenzo di Pasini Maiti (da Bergamo, mercante...",999768,Origine comune,NaN,NaN,NaN


In [ ]:
df_social[df_social.pk_processetto == 1007933]

,pk_processetto,pk_person,person,qualità,ruolo,pk_secondPerson,secondPerson,id_relazione,relation_type,begin,at_some_time,end
31,1007933.0,1007877,"Lucia di Antonio Rasonini (da Ampezzo, massera)",richiedente,partner,1007886.0,Antonio di Antonio Grappolino Bertolini (masser),1008021,Conoscenza in comune,1587.0,NaN,NaN
32,1007933.0,1007877,"Lucia di Antonio Rasonini (da Ampezzo, massera)",richiedente,partner,1007901.0,"Pietro di Antonio Meneghelli (da Ampezzo, mara...",1008028,Origine comune,NaN,NaN,NaN


--------------------------------

## Account

Importa unità di misura, crea dizionario, integra nel generatore di df la sostituzione chiave-valore

In [ ]:
file = 'geovistory_query_exports/nb_time_unit.json'
with open(file, encoding='utf-8') as json_file:
    data_time = json.load(json_file)
dmtu= DotMap(data_time)

dmtu_r = dmtu.rows
# Scarica nuovamente il json

dizionario = {}

for e in dmtu_r:
    dizionario[e.col_0.entity.pk_entity]  = e.col_0.entity.entity_label

dizionario


{922206: 'Week', 905203: 'Day', 905130: 'Year', 905137: 'Month'}

In [2]:
file = 'geovistory_query_exports/nb_account.json'

with open(file, encoding='utf-8') as json_file:
    data_account = json.load(json_file)

dmac = DotMap(data_account)
dmac_r = dmac.rows

acl = [['pk_processetto', 'id_person', 'person', 'id_account', 'len_account_type', 'acount_type', 'frequenza', 'concetti', 'geographical_place', 
'durata', 'unita', 'durata_s', 'unita_s', 'primo_timelaps', 'timelaps_unit' ,'secondo_timelaps', 'timelaps_unit_s']]
for e in dmac_r:
    for el in e.col_0.entities:
        element=[]
        element += [el.pk_entity, e.col_1.entities[0].pk_entity, clean_up_str(e.col_1.entities[0].entity_label), e.col_2.entity.pk_entity]
        tipi = []
        for ts in e.col_3.entities:
            tipi.append(ts.entity_label)
        element += [len(e.col_3.entities), '; '.join(tipi) if len(tipi)>=1 else np.nan, e.col_4.entities[0].entity_label if e.col_4.entities else np.nan]
        concetti=[]
        for conc in e.col_5.entities:
            concetti.append(clean_up_str(conc.entity_label))
        element +=[concetti if len(concetti)>=1 else np.nan]
        geo_p = []
        for gp in e.col_6.entities:
            geo_p.append(gp.entity_label)
        element+= [geo_p if len(geo_p)>=1 else np.nan, e.col_7['values'][0]['value'].dimension.numericValue if e.col_7['values'] else np.nan, 
        dizionario[e.col_7['values'][0]['value'].dimension.fkMeasurementUnit] if e.col_7['values'] else np.nan,
        e.col_7['values'][1]['value'].dimension.numericValue if len(e.col_7['values'])>1 else np.nan, 
        dizionario[e.col_7['values'][1]['value'].dimension.fkMeasurementUnit] if len(e.col_7['values'])>1 else np.nan, 
        e.col_8['values'][0]['value'].dimension.numericValue if e.col_8['values'] else np.nan, 
        dizionario[e.col_8['values'][0]['value'].dimension.fkMeasurementUnit]if e.col_8['values'] else np.nan, 
        e.col_9['values'][0]['value'].dimension.numericValue if e.col_9['values'] else np.nan, 
        dizionario[e.col_9['values'][0]['value'].dimension.fkMeasurementUnit]if e.col_9['values'] else np.nan]        
        acl.append(element)

df_account = pd.DataFrame(acl[1:], columns= acl[0])
df_account


NameError: name 'json' is not defined

In [1]:
df_account[df_account.pk_processetto.isnull()]

NameError: name 'df_account' is not defined